# Define Classes for Field and Term Objects


In [1]:
import numpy as np

In [2]:
class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq
    
class term(object):
    def __init__(self, field_list, lorentz_contractions=[], spinor_contractions=[]):
        self.field_list = field_list #ordered list of field objects
        self.lorentz_contractions = lorentz_contractions
        self.spinor_contractions = spinor_contractions #list of 2-tuples, indicating index values of fields in self.fields that are contracted 
    def __eq__(self, other):
        eq = (self.field_list == other.field_list)&(self.lorentz_contractions == other.lorentz_contractions)&(self.spinor_contractions == other.spinor_contractions)
        return eq
    def __str__(self):
        return 'fields: ' + str([item.get_symbol() for item in self.field_list]) + '\n' \
            + 'lorentz contractions: ' + str(self.lorentz_contractions) + '\n' \
            + 'spinor contractions: ' + str(self.spinor_contractions)
    def get_field_list(self):
        return self.field_list
    def get_lorentz_contractions(self):
        return self.lorentz_contractions
    def get_spinor_contractions(self):
        return self.spinor_contractions
    def set_field_list(self, field_list):
        self.field_list = field_list
    def set_lorentz_contractions(self, lorentz_contractions):
        self.lorentz_contractions = lorentz_contractions
    def set_spinor_contractions(self, spinor_contractions):
        self.spinor_contractions = spinor_contractions    
    def massDim(self):
        massDim = 0
        field_list = self.field_list
        #print(field_list)
        for field in field_list:
            massDim += field.get_massDim()
        return massDim 
    def get_field_symbols(self):
        field_symbols = []
        for item in self.field_list:
            field_symbols.append(item.get_symbol())
        return field_symbols    

In [3]:
#spacetime derivative
D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)

#dynamical fields
F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)

#16 spinor space blinears: 1 lorentz scalar, 4 lorentz vectors, 6 lorentz tensors, 4 lorentz pseudovector, 1 lorentz pseudoscalar. 
#not dynamical fields, constant matrices in spinor space
_S_ = field('_S_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
_V_ = field('_V_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
_T_ = field('_T_', massDim=0, lorentz_rank=2, spinor_rank=1, spinor_rank_conj=1) #6 of these
_Vp_ = field('_Vp_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
_Sp_ = field('_Sp_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these

# Generate All Terms of a Given Mass Dimension - with One Free Lorentz Index (for IBP), and with No Free Indices

*Preliminary thoughts*: Fix mass dimension, M. Fields in the term must be such that mass dimensions add up to M. Note that Dirac bilinear terms have no mass dimension, so the mass dimension does not directly constrain the allowed number of these. But constant matrices relate different index spaces (e.g., lorentz, spinor). 
- count number of derivatives up to and including 0, ..., M-1 (only M-1 and not M because a term must contain at least one dynamical field). 
- for each number of derivatives, find all combinations of fields consistent with the mass dimension (forgetting about contractions for the moment).
- for each number of derivatives, and for each such combination of fields, perform all possible contractions, including all combinations of constant bilinear spinor space matrices. 
- define an equivalence relation between terms (based on commutation/anticommutation of derivatives/fields, various identities such as Fierz identities) to remove repeats. 

## Generate All Combinations of Fields with a Given Mass Dimension

In [4]:
def generate_field_combos_d(massDim, num_derivs):
    #INPUT: mass dimension of field combo and number of derivatives in field combo
    #OUPTUT: list of lists, where each sublist is a field combo with num_derivs derivatives. 
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=1.5, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0) #Psi
    Pb = field('Pb', massDim=1.5, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1) #Psi_bar
    
    if massDim <= num_derivs:
        return []
    else:
        combo_root = num_derivs*[D]
        combo_roots_list = [combo_root]
        massDim_res_max = massDim - num_derivs
       
        
        while massDim_res_max >= 2:
            combo_roots_list_new = []
        
            for i in range(len(combo_roots_list)):
                massDim_root = sum([item.get_massDim() for item in combo_roots_list[i]])
                massDim_res = massDim - massDim_root
                #append any elements that already have residual massDim less than 2
                if massDim_res < 2:
                    combo_roots_list_new.append(combo_roots_list[i])
                if massDim_res >= 2:
                    combo_root_new1 = combo_roots_list[i].copy()
                    combo_root_new1.append(F)
                    combo_roots_list_new.append(combo_root_new1)
                if massDim_res >= 3:
                    combo_root_new2 = combo_roots_list[i].copy()
                    combo_root_new2.append(Pb)
                    combo_root_new2.append(P)
                    combo_roots_list_new.append(combo_root_new2)  
            
            massDim_res_max = max([(massDim - sum([item.get_massDim() for item in combo_roots_list_new[i]])) for i in range(len(combo_roots_list_new))])
            combo_roots_list = combo_roots_list_new
            
        combos_list_pruned = []
        for combo in combo_roots_list:
            combo_massDim = sum([item.get_massDim() for item in combo])
            if combo_massDim == massDim:
                combos_list_pruned.append(combo)
            else:
                pass    

    return combos_list_pruned


In [5]:
field_combos = generate_field_combos_d(6, 1)
print("len(field_combos): " + str(len(field_combos)))
for combo in field_combos:
    print(str([item.get_symbol() for item in combo]))

len(field_combos): 2
['D', 'F', 'Pb', 'P']
['D', 'Pb', 'P', 'F']


In [12]:
#NEW - missing FF term in dim4 lagragian from generate() below
field_combos = generate_field_combos_d(4, 0)
for combo in field_combos:
    print(str([item.get_symbol() for item in combo]))

    

['F', 'F']


In [13]:
def generate_field_combos(massDim):
    combos_list = []
    for num_derivs in range(massDim):
        combos_list_d = generate_field_combos_d(massDim, num_derivs)
        combos_list += combos_list_d
    return combos_list        

In [14]:
field_combos2 = generate_field_combos(6)
print("len(field_combos): " + str(len(field_combos2)))
for combo in field_combos2:
    print(str([item.get_symbol() for item in combo]))

len(field_combos): 7
['F', 'F', 'F']
['Pb', 'P', 'Pb', 'P']
['D', 'F', 'Pb', 'P']
['D', 'Pb', 'P', 'F']
['D', 'D', 'F', 'F']
['D', 'D', 'D', 'Pb', 'P']
['D', 'D', 'D', 'D', 'F']


In [15]:
def convert_to_symbol_list(field_list):
    symbol_list = [item.get_symbol() for item in field_list]
    return(symbol_list)

def convert_to_field_list(symbol_list):
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
    Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)
    
    _S_ = field('_S_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
    _V_ = field('_V_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
    _T_ = field('_T_', massDim=0, lorentz_rank=2, spinor_rank=1, spinor_rank_conj=1) #6 of these
    _Vp_ = field('_Vp_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
    _Sp_ = field('_Sp_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
    
    fields = [D, F, P, Pb, _S_, _V_, _T_, _Vp_, _Sp_]
    
    field_list = [Field for symbol in symbol_list for Field in fields if symbol == Field.get_symbol()]
    
    return field_list
                
def field_counts(symbol_list):
    D_count = 0
    F_count = 0
    PPb_count = 0 #Pb always comes with P, so no need to count Pb
    for item in symbol_list:
        if item == 'D':
            D_count += 1
        elif item == 'F':
            F_count += 1
        elif item == 'P':
            PPb_count += 1 
    field_counts = [D_count, F_count, PPb_count]
    return field_counts

def combos_equiv(symbol_list1, symbol_list2):
    return field_counts(symbol_list1) == field_counts(symbol_list2)



In [16]:
symbol_list = ['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P']
[item.get_symbol() for item in convert_to_field_list(symbol_list)]

['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P']

In [17]:
field_counts(convert_to_symbol_list(field_combos[11]))

IndexError: list index out of range

In [18]:
combos_equiv(convert_to_symbol_list(field_combos[10]), convert_to_symbol_list(field_combos[11]))

IndexError: list index out of range

Remove repeat field combos.

In [19]:
def reduce_field_combos(combo_list):
    #INPUT
    #combo_list: list of lists of field symbols (strings)
    #OUTPUT
    #combo_list_reduced: list of lists of field symbols, with repeat combos removed
    #EXPLANATION: removes repeat field combos from combo_list
    #print("len(combo_list): " + str(len(combo_list)))
    combo_list_reduced = [combo_list[0]]
    for i in range(len(combo_list)):
        #print("combo1: " + str(combo1))
        #print("")
        #print("i: " + str(i))
        #print("len(combo_list_reduced): " + str(len(combo_list_reduced)))
        for j in range(len(combo_list_reduced)):
            #print("combo2: " + str(combo2))
            #print("j: " + str(j))
            if combos_equiv(combo_list[i], combo_list_reduced[j]):
                #print("break")
                break
        else:
            combo_list_reduced.append(combo_list[i])
    return combo_list_reduced  
    
    

In [20]:
massDim = 6
symbol_lists = []
for combo in generate_field_combos(massDim):
    symbol_lists.append(convert_to_symbol_list(combo))
symbol_lists
    
combo_list_reduced = reduce_field_combos(symbol_lists)
len(combo_list_reduced)
combo_list_reduced

[['F', 'F', 'F'],
 ['Pb', 'P', 'Pb', 'P'],
 ['D', 'F', 'Pb', 'P'],
 ['D', 'D', 'F', 'F'],
 ['D', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'F']]

In [21]:
#NEW - debug generate()
massDim = 4
symbol_lists = []
for combo in generate_field_combos(massDim):
    symbol_lists.append(convert_to_symbol_list(combo))
symbol_lists
    
combo_list_reduced = reduce_field_combos(symbol_lists)
len(combo_list_reduced)
combo_list_reduced


[['F', 'F'], ['D', 'Pb', 'P'], ['D', 'D', 'F']]

In [22]:
def generate_field_combos_reduced(massDim):
    symbol_lists = []
    for combo in generate_field_combos(massDim):
        symbol_lists.append(convert_to_symbol_list(combo))
    field_combos_reduced = reduce_field_combos(symbol_lists)
    return field_combos_reduced

massDim = 6
generate_field_combos_reduced(massDim)

[['F', 'F', 'F'],
 ['Pb', 'P', 'Pb', 'P'],
 ['D', 'F', 'Pb', 'P'],
 ['D', 'D', 'F', 'F'],
 ['D', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'F']]

## Generate Derivative Assignments for Each Combination of Fields

In [23]:
#helper functions for generate_derivative_assignments
def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def sums_to_n(n, m): # generates partitions of derivatives to each field
    #INPUT
    #n: number we want to sum to; i.e. total number of derivatives
    #m: number of digits in sum; i.e. total number of fields
    #OUTPUT
    #list of lists of digits, with each sublist no longer than m
    #EXPLANATION
    #list of combos for n can be generated from list of combos for n-1 either by appending 1 to each of the combos
    #for n-1, or adding 1 to just one of the elements in each combo. To ensure no repeats, we require the lists to be
    #non-increasing.
    combos_list = []
    if n==0:
        combo = m*[0]
        combos_list.append(combo)
        return combos_list
    elif n==1:
        combo = [1] + (m-1)*[0]
        combos_list.append(combo)
        return combos_list
    elif m==0:
        return []
    else:
        combos_list_prev = sums_to_n(n-1, n-1)
        for combo in combos_list_prev:
            combo_new1 = combo + [1]
            if non_increasing(combo_new1) and combo_new1 not in combos_list:
                combos_list.append(combo_new1) 
            for i in range(len(combo)):
                combo_new2 = combo.copy()
                combo_new2[i] += 1
                if non_increasing(combo_new2) and combo_new2 not in combos_list:
                    combos_list.append(combo_new2) 
        combos_list_trunc = [combo for combo in combos_list if len(combo) <= m]    
        return combos_list_trunc

def pad_w_zeros(list_of_lists, size):
    #INPUT
    #list_of_lists: list of lists of numbers; all sublists should have length less than or equal to size
    #size: int, common size of all lists after padding
    #OUTPUT
    #list_of_lists_padded: padded list of lists, where each sublist has length size
    
    if size < max([len(sublist) for sublist in list_of_lists]):
        print("'size' input must be larger than or equal max length of sublists in 'list_of_lists' input.")
        return None
    else:
        list_of_lists_padded = []
        for sublist in list_of_lists:
            num_zeros = size - len(sublist)
            sublist_padded = sublist + num_zeros*[0]
            list_of_lists_padded.append(sublist_padded)
        return list_of_lists_padded
        
def sums_to_n_padded(n, m):
    return pad_w_zeros(sums_to_n(n, m), m) 

def generate_derivative_assignments_recursively(field_symbol_list, num_fields_list, d):
    #INPUT
    #field_symbol_list: list of field symbols - e.g., ['F', 'Pb', 'P']
    #num_fields_list: number of each type of non-derivative field [num_F, num_Pb, num_P], must be same length
    #as field_symbol_list
    #d: total number of derivatives acting on all fields
    #OUTPUT
    #derivative_assignments_list: list of lists, with each sublist indicating the number of derivatives assigned 
    #to each (non-derivative) field. number of derivatives preceding each field indicates number of derivatives
    #acting on that field: e.g. [['D', 'D', 'F', 'D', 'F', 'D', 'D', 'D', 'Pb', 'D', 'P', 'D', 'D', 'P'],
    #['D', 'F', 'D', 'Pb', 'D', 'D', 'D', 'Pb', 'D', 'D', 'P'], ...]
    
    #find index of last non-zero term in num_fields_list. this will serve as a base case for recursion
    last_nonzero_index = None #initialize last_nonzero_index 
    for i in range(1, len(num_fields_list)+1): #find index (from end of list) of last non-zero term in num_fields_list; deposit all 
        #remaining derivatives with this field type.
        if num_fields_list[-i]!=0:
            last_nonzero_index = -i
            break
    #print("last_nonzero_index: " + str(last_nonzero_index))
    if len(field_symbol_list)!=len(num_fields_list):
        print("field_symbol_list and num_fields_list must have same length.")
        return []
    if num_fields_list==[] or field_symbol_list==[]:
        print("num_fields_list and field_symbol_list cannot be empty.")
        return []
    if d==0:
        fields = []
        for i in range(len(field_symbol_list)):
            field_symbol = field_symbol_list[i]
            for j in range(num_fields_list[i]):
                fields.append(field_symbol)
        return [fields]
    elif num_fields_list[0]==0: #if no fields of a particular type, move on to next type
        return generate_derivative_assignments_recursively(field_symbol_list[1:], num_fields_list[1:], d)
    elif len(num_fields_list)==-last_nonzero_index: #if we are on the last field type with non-zero entry in 
        #print("last non-zero entry in num_fields_list")
        n_0 = num_fields_list[0]
        field_symbol = field_symbol_list[0]
        derivative_assignments_list = []
        derivs_0 = sums_to_n_padded(d, n_0)
        appended_list = []
        for num_derivs_list in derivs_0:
            appended = []
            for j in num_derivs_list:
                appended += j*['D']
                appended += [field_symbol]
            appended_list.append(appended)
        return appended_list
    else:
        n_0 = num_fields_list[0]
        field_symbol = field_symbol_list[0]
        derivative_assignments_list = []
        #for each possible number of derivatives acting on field species corresponding to 0th index of num_fields_list,
        #compute all possible ways of distributing derivatives among those fields. 
        for d_0 in range(d+1):
            derivs_0 = sums_to_n_padded(d_0, n_0)
            #generate a list of sublists of symbols associated with each element of derivs_0 - e.g., one such  
            #sublist might be ['D', 'D', 'F', 'D', 'F', 'D', 'D', 'D', 'F']
            appended_list = []
            for num_derivs_list in derivs_0:
                appended = []
                for j in num_derivs_list:
                    appended += j*['D']
                    appended += [field_symbol]
                    #print("field_symbol: " + str(field_symbol))
                    #print("appended: " + str(appended))
                appended_list.append(appended)
            #print("field_symbol_list[1:]: " + str(field_symbol_list[1:]))
            derivative_assignments_list_old = generate_derivative_assignments_recursively(field_symbol_list[1:], num_fields_list[1:], d-d_0)
            for appended in appended_list:
                for derivative_assignment_old in derivative_assignments_list_old:
                    derivative_assignment_new = appended + derivative_assignment_old
                    derivative_assignments_list.append(derivative_assignment_new)
        return derivative_assignments_list

    
def generate_derivative_assignments(field_combo):
    #INPUT
    #field_combo: List of string symbols for derivatives and non-derivative fields (called simply fields here).  
    #OUTPUT
    #derivative_assigments: list of lists, where each sublist contains a distinct assignment of derivatives in
    #field_combo to fields in field_combo
    #EXPLANATION: generates all assignments of derivatives to fields for a given combination. 
    d = 0 #initialize number of derivatives
    n_F = 0 #initialize number of F fields
    n_P = 0 #initialize number of P fields
    derivative_assigment_list = [] #lists of lists, where the elements of each sublist are field symbols; the 
    #number of 'D' symbols preceding a given non-derivative field indicates the number of derivatives acting on 
    #that field.
    for item in field_combo:
        if item == 'D':
            d += 1
        elif item == 'F':
            n_F += 1
        elif item == 'P':
            n_P += 1
        #no need to count Pb since there are the same number of these as of P
    field_symbol_list = ['F', 'Pb', 'P']
    num_fields_list = [n_F, n_P, n_P]
    return generate_derivative_assignments_recursively(field_symbol_list, num_fields_list, d)


In [24]:
field_symbol_list = ['F', 'Pb', 'P']
num_fields_list = [1, 5, 5] 
d = 4 
generate_derivative_assignments_recursively(field_symbol_list, num_fields_list, d)

[['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'P',
  'D',
  'D',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'D',
  'D',
  'P',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'D',
  'P',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P',
  'P'],
 

In [25]:
field_combo = ['D', 'D', 'D', 'F', 'F', 'Pb', 'P']
generate_derivative_assignments(field_combo)

[['F', 'F', 'Pb', 'D', 'D', 'D', 'P'],
 ['F', 'F', 'D', 'Pb', 'D', 'D', 'P'],
 ['F', 'F', 'D', 'D', 'Pb', 'D', 'P'],
 ['F', 'F', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'F', 'F', 'Pb', 'D', 'D', 'P'],
 ['D', 'F', 'F', 'D', 'Pb', 'D', 'P'],
 ['D', 'F', 'F', 'D', 'D', 'Pb', 'P'],
 ['D', 'F', 'D', 'F', 'Pb', 'D', 'P'],
 ['D', 'F', 'D', 'F', 'D', 'Pb', 'P'],
 ['D', 'D', 'F', 'F', 'Pb', 'D', 'P'],
 ['D', 'D', 'F', 'F', 'D', 'Pb', 'P'],
 ['D', 'D', 'F', 'D', 'F', 'Pb', 'P'],
 ['D', 'D', 'D', 'F', 'F', 'Pb', 'P']]

## Generate All Full and One-Free-Lorentz-Index Contractions from a Given Derivative Assignment

### Contracting Spinor Indices

Consider a term of the form

$(D_{\mu^{1}_{1}} ... D_{\mu^{1}_{k_{1}}} \bar{\psi}) ... (D_{\mu^{n}_{1}} ... D_{\mu^{n}_{k_{n}}} \bar{\psi}) (D_{\nu^{1}_{1}} ... D_{\nu^{1}_{l_{1}}} \psi) ... (D_{\nu^{n}_{1}} ... D_{\nu^{n}_{l_{n}}} \psi) (D_{\alpha_{1}} ... D_{\alpha_{p}} F_{\kappa \gamma})$.

There are the same number of $\bar{\psi}$ as $\psi$. Any (D... $\bar{\psi}$) can be contracted in spinor space with any  (D... $\psi$). There are $n$ of the first and $n$ of the second, so $n^2$ altogether. For each distinct $A_{i} = D_{\mu^{i}_{1}} ... D_{\mu^{i}_{k_{1}}} \bar{\psi}$ and $B_{j} = D_{\nu^{j}_{1}} ... D_{\nu^{j}_{l_{j}}} \psi$, we have 

$A_{i} M B_{j}$, 

where $M \in \{I, \gamma_{\mu}, \sigma_{\mu \nu}, \gamma_{5}\gamma_{\mu}, \gamma_{5} \}$. Thus, we have 5 possibilities for each pair of $\bar{\psi}$ and $\psi$. For each of the $n^2$ terms, we have $5^{n}$ distinct possibilities. So, including the different matrices in $M$, we generate

$n^{2}5^{n}$

terms. Write a function, *generate_spinor_contractions(derivative_assignment)*, which takes a derivative assignment and returns all such $n^{2}5^{n}$ terms. 





In [26]:
def generate_spinor_contractions_from_indices(Pb_indices, P_indices):
    #INPUT
    #P_indices: list of locations in derivative_assignment (see generate_spinor_contractions) where P occurs
    #Pb_indices: same for Pb
    #EXPLANATION
    #Generates all possible pairings of spinor fields
    #Check that lists are of same length
    if len(Pb_indices) != len(P_indices):
        print('Pb_indices, P_indices must be of same length.')
        return None
    elif len(Pb_indices)==1: #base case
        list_of_contraction_lists = [[(Pb_indices[0], P_indices[0])]]
        return list_of_contraction_lists
    else:
        n = len(Pb_indices)
        list_of_contraction_lists = [] #store lists of contractions of Pb with P; each of length n
        for j in range(n):
            contraction = (Pb_indices[0], P_indices[j]) #start with first index for Pb_indices; as it is deleted,
            #the recursion will carry us through each element of Pb_indices and cover all pairs of Pb and P just once. 
            Pb_indices_copy = Pb_indices.copy()
            P_indices_copy = P_indices.copy()

            del(Pb_indices_copy[0])
            del(P_indices_copy[j])

            for contraction_list in generate_spinor_contractions_from_indices(Pb_indices_copy, P_indices_copy):
                contraction_list_new = [contraction] + contraction_list
                list_of_contraction_lists.append(contraction_list_new)          

    return list_of_contraction_lists

def generate_spinor_contractions(derivative_assignment):
    #EXPLANATION
    #Generates all possible pairings of contracted spinor fields (by position of spinor fields) in derivative_assignment
    #separate out terms of the form D...DP and D...DPb, or P, Pb. 
    Pb_indices = [] #collect Pb's, Pb derivatives
    P_indices = [] #collect P's, P derivatives
    spinor_contractions = [] #list of 2-tuples
    #find indices with Pb or P
    for i in range(len(derivative_assignment)):
        if derivative_assignment[i]=='Pb':
            Pb_indices.append(i)
        if derivative_assignment[i]=='P':
            P_indices.append(i)
    
    list_of_contraction_lists = generate_spinor_contractions_from_indices(Pb_indices, P_indices)
        
    return list_of_contraction_lists


In [27]:
#derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'P', 'P']
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
spinor_contractions = generate_spinor_contractions(derivative_assignment)
spinor_contractions

[[(1, 6), (2, 8), (3, 10)],
 [(1, 6), (2, 10), (3, 8)],
 [(1, 8), (2, 6), (3, 10)],
 [(1, 8), (2, 10), (3, 6)],
 [(1, 10), (2, 6), (3, 8)],
 [(1, 10), (2, 8), (3, 6)]]

For each contraction of a $\bar{\Psi}$ with a $\Psi$, we can insert any of the bilinears $M = \{I, \gamma_{\mu}, \sigma_{\mu \nu}, \gamma_{5}\gamma_{\mu}, \gamma_{5} \}$ to get $\left(D_{\mu_{1}} ... D_{\mu_{k}}\bar{\Psi} \right)\left(M\right) \left(D_{\nu_{1}} ... D_{\nu_{l}}\Psi \right)$. For each 

In [28]:
def generate_bilinear_contractions_from_single_list(contraction_list):
    #bilinears
    M = ['_S_', '_V_', '_T_', '_Vp_', '_Sp_']
    #base case
    if len(contraction_list)==1:
        list_of_bilinear_contraction_lists = []
        for m in M:
            contraction_list_new =  [(contraction_list[0], m)]
            list_of_bilinear_contraction_lists.append(contraction_list_new)
        return list_of_bilinear_contraction_lists
    else:
        list_of_bilinear_contraction_lists = []
        for m in M:
            bilinear_contraction = (contraction_list[0], m)
            for contraction_list_trunc in generate_bilinear_contractions_from_single_list(contraction_list[1:]):
                contraction_list_new = [bilinear_contraction] + contraction_list_trunc
                list_of_bilinear_contraction_lists.append(contraction_list_new)
        return list_of_bilinear_contraction_lists
    
def generate_bilinear_contractions(derivative_assignment):
    #EXPLANATION
    #Generates all possible spinor contractions with all possible bilinears
    list_of_contraction_lists = generate_spinor_contractions(derivative_assignment)
    list_of_bilinear_contraction_lists = []
    for contraction_list in list_of_contraction_lists:
        list_of_bilinear_contraction_lists += generate_bilinear_contractions_from_single_list(contraction_list)
    return list_of_bilinear_contraction_lists
    
 

In [29]:
contraction_list = [(1, 6), (2, 8), (3, 10)]
generate_bilinear_contractions_from_single_list(contraction_list)

[[((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_T_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_Vp_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_Sp_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_T_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_Vp_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_Sp_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_T_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_Vp_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_Sp_')],
 [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_T_')],
 

In [30]:
len(generate_bilinear_contractions(derivative_assignment))

750

### Remove Redundant Spinor Contractions

Next, remove redundant spinor contractions. Two spinor contractions are equivalent iff 

- they have the same number $n$ of $\bar{\psi}$ and $\psi$. 
- they have the same set $\{(d_{1}, d_{2}, m)\}$ of $n$ triplets, where $d_{1}$ is the number of derivatives acting on $\bar{\psi}$, $d_{2}$ is the number of derivatives acting on $\psi$, and $m \in M$, or each pair of $\bar{\psi}$ and $\psi$.

To do this, first write a function *equiv_spinor_contractions(spinor_contraction1, spinor_contraction2)* that takes two spinor contractions (with all Lorentz indices uncontracted), and outputs True if they are equivalent and False if not. Then write a function *reduce_spinor_contraction_list(spinor_contraction_list)* that takes a list of spinor contractions and returns that list with repetitions removed. 

In [31]:
def spinor_contractions_equiv(spinor_contraction1, spinor_contraction2):
    #INPUT
    #-spinor_contraction1: a list with 2 elements - 1) a list of field symbols indicating derivative assignments to different 
    #fields, assumed ordered with F's to the left of Pb's to the left of P's, 
    #2) a list of tuples indicating the locations of contracted Pb and P in the derivative assignment list,
    #as well as the bilinear matrix with which they are contracted. 
    #-spinor_contraction2: same as spinor_contraction1
    #OUTPUT
    #equiv: True if the spinor contractions are equivalent and False otherwise. 
    #EXPLANATION
    #once it has been checked that the derivative assignments are the same, for each Pb/P contraction, identify a 
    #tuple of 1) the bilinear matrix, 2) the number of derivatives acting on Pb, 3) the number of derivatives acting 
    #on P. for spinor_contraction1 and spinor_contraction2 to be equivalent, the set of such tuples for both should
    #be the same. 
    
    #check that derivative assignment lists are the same
    derivative_assignment1 = spinor_contraction1[0]
    derivative_assignment2 = spinor_contraction2[0]
    
    if derivative_assignment1 != derivative_assignment2:
        return False 
    
    #check that the numbers of Pb/P contractions are the same
    bilinear_contraction_list1 = spinor_contraction1[1]
    bilinear_contraction_list2 = spinor_contraction2[1]
    
    if len(bilinear_contraction_list1) != len(bilinear_contraction_list2):
        return False
    
    #Extract set of (n_d, n_db, M) triples for spinor_contraction1 and spinor_contraction2
    triples_list1 = extract_derivative_bilinear_triples(spinor_contraction1)
    triples_list2 = extract_derivative_bilinear_triples(spinor_contraction2)
    
    if set(triples_list1) == set(triples_list2):
        equiv = True
    else:
        equiv = False

    return equiv


def extract_derivative_bilinear_triples(spinor_contraction):
    #INPUT
    #-spinor_contraction: a list with 2 elements - 1) a list of field symbols indicating derivative assignments to different 
    #fields, assumed ordered with F's to the left of Pb's to the left of P's, 
    #2) a list of tuples indicating the locations of contracted Pb and P in the derivative assignment list,
    #as well as the bilinear matrix with which they are contracted. 
    #OUTPUT
    #triples_list: a list of 3-tuples (n_db, n_d, M), one for each Pb/P pair in spinor_contraction. for each 
    #Pb/P pair, the corresponding 3-tuple indicates the number n_db of derivatives acting on the Pb, n_d the
    #number of derivatives acting on the P, and M the bilinear matrix sandwiched between them. 
    
    #extract derivative assignment list and bilinear contraction list
    derivative_assignment = spinor_contraction[0]
    bilinear_contraction_list = spinor_contraction[1]
    
    triples_list = [] #store set of triples (n_dp, n_d, M), one for each Pb/P contraction. 
    #count number of derivatives n_dp and n_p acting on each Pb and P, respectively. 
    for i in range(len(bilinear_contraction_list)):
        #unpack bilinear contraction
        bilinear_contraction = bilinear_contraction_list[i]
        contraction_indices = bilinear_contraction[0]
        M = bilinear_contraction[1]
        P_index = contraction_indices[1]
        Pb_index = contraction_indices[0]
        
        #count number of D's acting on field P at P_index in derivative_assignment
        n_d = 0
        i = 1
        while P_index - i >= 0 and derivative_assignment[P_index - i] == 'D':
            n_d += 1
            i += 1
            
        #count number of D's acting on field Pb at Pb_index in derivative_assignment
        n_db = 0
        i = 1
        while Pb_index - i >= 0 and derivative_assignment[Pb_index - i] == 'D':
            n_db += 1
            i += 1
        
        triple = (n_db, n_d, M)
        triples_list.append(triple)
    
    return triples_list
    

In [32]:
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
bilinear_contraction_list1 = [((1, 6), '_V_'), ((2, 8), '_V_'), ((3, 10), '_T_')]
bilinear_contraction_list2 = [((1, 6), '_V_'), ((2, 10), '_V_'), ((3, 8), '_T_')]
#bilinear_contraction_list2 = [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_S_')]

spinor_contraction1 = [derivative_assignment, bilinear_contraction_list1]
spinor_contraction2 = [derivative_assignment, bilinear_contraction_list2]

print(spinor_contractions_equiv(spinor_contraction1, spinor_contraction2))

print(extract_derivative_bilinear_triples(spinor_contraction1))
print(extract_derivative_bilinear_triples(spinor_contraction2))

True
[(0, 2, '_V_'), (0, 1, '_V_'), (0, 1, '_T_')]
[(0, 2, '_V_'), (0, 1, '_V_'), (0, 1, '_T_')]


In [33]:
def reduced_spinor_contractions(derivative_assignment):
    #INPUT
    #derivative_assignment: list of field symbols where the number of 'D' preceding a field indicates the number
    #of derivatives acting on that field
    #OUTPUT
    #reduced_list: list of inequivalent bilinear spinor contractions associated with a particular derivative assignment
    
    #generate all bilinear spinor contractions
    list_of_bilinear_contractions = generate_bilinear_contractions(derivative_assignment)
    #initialize empty list for storing unique elements
    reduced_list = []
    
    for bilinear_contraction in list_of_bilinear_contractions:
        include_in_reduced_list = True
        #print("bilinear_contraction: " + str(bilinear_contraction))
        for item in reduced_list: #could make more efficient here; shouldn't need to loop through all elements of reduced_list for each bilinear contraction
            spinor_contraction1 = [derivative_assignment, bilinear_contraction]
            spinor_contraction2 = [derivative_assignment, item]
            if spinor_contractions_equiv(spinor_contraction1, spinor_contraction2):
                #print("equivalent item in reduced_list: " + str(item))
                include_in_reduced_list = False
                break
        if include_in_reduced_list:
            #print("append: " + str(bilinear_contraction))
            reduced_list.append(bilinear_contraction)
        
    return reduced_list

In [34]:
derivative_assignment = ['F', 'D', 'D', 'Pb', 'D', 'D', 'D', 'Pb', 'P', 'D', 'P']
reduced_list = reduced_spinor_contractions(derivative_assignment)
full_list = generate_bilinear_contractions(derivative_assignment)
for item in reduced_list:
    print(item)
'''
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
reduced_list = reduce_spinor_contractions(derivative_assignment)
full_list = generate_bilinear_contractions(derivative_assignment)
for item in reduced_list:
    print(item)
'''

[((3, 8), '_S_'), ((7, 10), '_S_')]
[((3, 8), '_S_'), ((7, 10), '_V_')]
[((3, 8), '_S_'), ((7, 10), '_T_')]
[((3, 8), '_S_'), ((7, 10), '_Vp_')]
[((3, 8), '_S_'), ((7, 10), '_Sp_')]
[((3, 8), '_V_'), ((7, 10), '_S_')]
[((3, 8), '_V_'), ((7, 10), '_V_')]
[((3, 8), '_V_'), ((7, 10), '_T_')]
[((3, 8), '_V_'), ((7, 10), '_Vp_')]
[((3, 8), '_V_'), ((7, 10), '_Sp_')]
[((3, 8), '_T_'), ((7, 10), '_S_')]
[((3, 8), '_T_'), ((7, 10), '_V_')]
[((3, 8), '_T_'), ((7, 10), '_T_')]
[((3, 8), '_T_'), ((7, 10), '_Vp_')]
[((3, 8), '_T_'), ((7, 10), '_Sp_')]
[((3, 8), '_Vp_'), ((7, 10), '_S_')]
[((3, 8), '_Vp_'), ((7, 10), '_V_')]
[((3, 8), '_Vp_'), ((7, 10), '_T_')]
[((3, 8), '_Vp_'), ((7, 10), '_Vp_')]
[((3, 8), '_Vp_'), ((7, 10), '_Sp_')]
[((3, 8), '_Sp_'), ((7, 10), '_S_')]
[((3, 8), '_Sp_'), ((7, 10), '_V_')]
[((3, 8), '_Sp_'), ((7, 10), '_T_')]
[((3, 8), '_Sp_'), ((7, 10), '_Vp_')]
[((3, 8), '_Sp_'), ((7, 10), '_Sp_')]
[((3, 10), '_S_'), ((7, 8), '_S_')]
[((3, 10), '_S_'), ((7, 8), '_V_')]
[((3, 10

"\nderivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']\nreduced_list = reduce_spinor_contractions(derivative_assignment)\nfull_list = generate_bilinear_contractions(derivative_assignment)\nfor item in reduced_list:\n    print(item)\n"

### Contracting Lorentz Indices

Next, for each spinor contraction, generate all possible contractions of Lorentz indices. For an even number of Lorentz indices, generate all ways of separating them into pairs. 

For an odd number of Lorentz indices, each index may serve as the free index; for each such choice of the free index, generate all possible groupings into pairs of the remaining even number of Lorentz indices; these one-Lorentz-index terms will serve to generate the integration by parts (IBP) relations. 

In [35]:
def convert_to_term_object(derivative_assignment, bilinear_contraction_list):
    #INPUT
    #-derivative_assignment: list of symbols (D, F, P, Pb) denoting field objects. number of D's immediately preceding
    #an F, P, or Pb designates the number of derivatives acting on that field. 
    #-bilinear_contraction_list: list of tuples ((i, j), M) indicating contractions of Pb's with P's, and the bilinear
    #matrix M sandwiched between them. 
    #OUTPUT
    #-term_object: a term object with field list and spinor contractions dictated by inputs, but lorentz 
    #contractions unspecified.
    
    #construct field symbol list with bilinears inserted
    derivative_assignment_shifted = derivative_assignment.copy() #for insertion of bilinears
    shift = 0 #indices are shifted when we insert bilinears into field symbol list. keep track of shift. 
    for i in range(len(derivative_assignment)):
        if derivative_assignment[i]=='Pb':
            #move D...DP with which Pb is contracted to the immediate right of Pb
            M = None
            for j in range(len(bilinear_contraction_list)):
                if bilinear_contraction_list[j][0][0]==i:
                    M = bilinear_contraction_list[j][1]
                    shift += 1
                    break
            derivative_assignment_shifted = derivative_assignment_shifted[:i+shift] + [M] + derivative_assignment_shifted[i+shift:]

    #determine indices in derivative_assignment where bilinears are inserted, i.e., where there are Pb's
    Pb_indices = []
    for i in range(len(derivative_assignment)):
        if derivative_assignment[i]=='Pb':
            Pb_indices.append(int(i))
    
    #extract spinor contractions from bilinear_contraction_list for adjustment of contraction indices after 
    #insertion of bilinears
    spinor_contractions = []
    for item in bilinear_contraction_list:
        #print('type(item[0][0]): ' + str(type(item[0])))
        spinor_contractions += [item[0]]
    #print('spinor_contractions: ' + str(spinor_contractions))
    
    #fill in shifted list of spinor contractions
    spinor_contractions_shifted = []
    for contraction in spinor_contractions:
        contraction_new = list(contraction)
        for j in range(len(Pb_indices)):
            if Pb_indices[j] < contraction[0]:
                #print('Pb_indices[j] < contraction[0]; j=' + str(j))
                contraction_new[0] = contraction[0] + j + 1
            if Pb_indices[j] < contraction[1]:
                #print('Pb_indices[j] < contraction[1]; j=' + str(j))
                contraction_new[1] = contraction[1] + j + 1
        #split contraction: Pb contracts with bilinear one step to its right, and bilinear contracts with P. 
        contraction_new_split = [(contraction_new[0], contraction_new[0]+1), (contraction_new[0]+1, contraction_new[1])]
        spinor_contractions_shifted += contraction_new_split
        #spinor_contractions_shifted.append(tuple(contraction_new))
    #print('spinor_contractions_shifted: ' + str(spinor_contractions_shifted))
    
    #convert derivative_assignment_shifted to list of field objects
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
    Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)
    _S_ = field('_S_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
    _V_ = field('_V_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
    _T_ = field('_T_', massDim=0, lorentz_rank=2, spinor_rank=1, spinor_rank_conj=1) #6 of these
    _Vp_ = field('_Vp_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
    _Sp_ = field('_Sp_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
    
    field_list = []
    for symbol in derivative_assignment_shifted:
        if symbol=='D':
            field_list.append(D)
        elif symbol=='F':
            field_list.append(F)
        elif symbol=='P':
            field_list.append(P)
        elif symbol=='Pb':
            field_list.append(Pb)
        elif symbol=='_S_':
            field_list.append(_S_)
        elif symbol=='_V_':
            field_list.append(_V_)
        elif symbol=='_T_':
            field_list.append(_T_)
        elif symbol=='_Vp_':
            field_list.append(_Vp_)
        elif symbol=='_Sp_':
            field_list.append(_Sp_)
    
    #instantiate term object
    term_object = term(field_list=field_list, lorentz_contractions=[], spinor_contractions=spinor_contractions_shifted) 
             
    return term_object
    

In [36]:
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
bilinear_contraction_list = [((1, 6), '_V_'), ((2, 8), '_V_'), ((3, 10), '_T_')]
term_object = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
print(term_object.get_field_symbols())
print(term_object.get_spinor_contractions())

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]


In [37]:
derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'D', 'P', 'P']
#derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
spinor_contractions = generate_spinor_contractions(derivative_assignment)
print(spinor_contractions)
reduced_list = reduced_spinor_contractions(derivative_assignment)
for item in reduced_list:
    print(item)

[[(5, 8), (6, 9)], [(5, 9), (6, 8)]]
[((5, 8), '_S_'), ((6, 9), '_S_')]
[((5, 8), '_S_'), ((6, 9), '_V_')]
[((5, 8), '_S_'), ((6, 9), '_T_')]
[((5, 8), '_S_'), ((6, 9), '_Vp_')]
[((5, 8), '_S_'), ((6, 9), '_Sp_')]
[((5, 8), '_V_'), ((6, 9), '_S_')]
[((5, 8), '_V_'), ((6, 9), '_V_')]
[((5, 8), '_V_'), ((6, 9), '_T_')]
[((5, 8), '_V_'), ((6, 9), '_Vp_')]
[((5, 8), '_V_'), ((6, 9), '_Sp_')]
[((5, 8), '_T_'), ((6, 9), '_S_')]
[((5, 8), '_T_'), ((6, 9), '_V_')]
[((5, 8), '_T_'), ((6, 9), '_T_')]
[((5, 8), '_T_'), ((6, 9), '_Vp_')]
[((5, 8), '_T_'), ((6, 9), '_Sp_')]
[((5, 8), '_Vp_'), ((6, 9), '_S_')]
[((5, 8), '_Vp_'), ((6, 9), '_V_')]
[((5, 8), '_Vp_'), ((6, 9), '_T_')]
[((5, 8), '_Vp_'), ((6, 9), '_Vp_')]
[((5, 8), '_Vp_'), ((6, 9), '_Sp_')]
[((5, 8), '_Sp_'), ((6, 9), '_S_')]
[((5, 8), '_Sp_'), ((6, 9), '_V_')]
[((5, 8), '_Sp_'), ((6, 9), '_T_')]
[((5, 8), '_Sp_'), ((6, 9), '_Vp_')]
[((5, 8), '_Sp_'), ((6, 9), '_Sp_')]
[((5, 9), '_S_'), ((6, 8), '_S_')]
[((5, 9), '_S_'), ((6, 8), '_V_')

In [38]:
derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'D', 'P', 'P']
bilinear_contraction_list = [((5, 8), '_T_'), ((6, 9), '_S_')]
term_object = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
print(term_object.get_field_symbols())
print(term_object.get_spinor_contractions())

['D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P']
[(5, 6), (6, 10), (7, 8), (8, 11)]


In [39]:
derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'D', 'P', 'P']
spinor_contracted_term = convert_to_term_object(derivative_assignment, bilinear_contraction_list)

#helper function for scalar_lorentz_contractions_from_spinor_contraction(spinor_contracted_term) below
def generate_pair_partitions(input_list):
    #EXPLANATION
    #generates all ways of partitioning the input list into pairs. if number of elements in list is odd,
    #generates all ways of partitioning the input list into pairs with one left over. 
    
    #recursion base case 1
    if len(input_list)==0:
        #print('BASE CASE, RETURN []')
        return [[]]
    elif len(input_list)%2==0:
        partition_list = [] #list of lists for collecting different partitions into pairs
        #input_list_copy = input_list.copy()
        #print('INPUT_LIST: ' + str(input_list))
        item1 = input_list[0]
        for j in range(1, len(input_list)):
            #print('i: ' + str(i))
            #print('j: ' + str(j))
            item2 = input_list[j]
            pair = (item1, item2)
            input_list_new = [input_list[k] for k in range(len(input_list)) if k not in [0,j]]
            #print('INPUT_LIST_NEW: ' + str(input_list_new))
            #print('INPUT_LIST_NEW, Post deletions: ' + str(input_list_new))
            for partition in generate_pair_partitions(input_list_new):
                partition_new = [pair] + partition
                partition_list.append(partition_new)
        return partition_list
    elif len(input_list)%2!=0:
        partition_list = []
        for i in range(len(input_list)):
            single = input_list[i]
            input_list_copy = input_list.copy()
            del input_list_copy[i]
            for partition in generate_pair_partitions(input_list_copy):
                #partition_new = [single] + partition #includes single uncontracte index in list
                #partition_list.append(partition_new)
                partition_list.append(partition)
        return partition_list         
            

In [40]:
input_list = [1,2,3,4,5,6,7]
generate_pair_partitions(input_list)

[[(2, 3), (4, 5), (6, 7)],
 [(2, 3), (4, 6), (5, 7)],
 [(2, 3), (4, 7), (5, 6)],
 [(2, 4), (3, 5), (6, 7)],
 [(2, 4), (3, 6), (5, 7)],
 [(2, 4), (3, 7), (5, 6)],
 [(2, 5), (3, 4), (6, 7)],
 [(2, 5), (3, 6), (4, 7)],
 [(2, 5), (3, 7), (4, 6)],
 [(2, 6), (3, 4), (5, 7)],
 [(2, 6), (3, 5), (4, 7)],
 [(2, 6), (3, 7), (4, 5)],
 [(2, 7), (3, 4), (5, 6)],
 [(2, 7), (3, 5), (4, 6)],
 [(2, 7), (3, 6), (4, 5)],
 [(1, 3), (4, 5), (6, 7)],
 [(1, 3), (4, 6), (5, 7)],
 [(1, 3), (4, 7), (5, 6)],
 [(1, 4), (3, 5), (6, 7)],
 [(1, 4), (3, 6), (5, 7)],
 [(1, 4), (3, 7), (5, 6)],
 [(1, 5), (3, 4), (6, 7)],
 [(1, 5), (3, 6), (4, 7)],
 [(1, 5), (3, 7), (4, 6)],
 [(1, 6), (3, 4), (5, 7)],
 [(1, 6), (3, 5), (4, 7)],
 [(1, 6), (3, 7), (4, 5)],
 [(1, 7), (3, 4), (5, 6)],
 [(1, 7), (3, 5), (4, 6)],
 [(1, 7), (3, 6), (4, 5)],
 [(1, 2), (4, 5), (6, 7)],
 [(1, 2), (4, 6), (5, 7)],
 [(1, 2), (4, 7), (5, 6)],
 [(1, 4), (2, 5), (6, 7)],
 [(1, 4), (2, 6), (5, 7)],
 [(1, 4), (2, 7), (5, 6)],
 [(1, 5), (2, 4), (6, 7)],
 

In [41]:
def lorentz_contractions_from_spinor_contraction(spinor_contracted_term):
    
    #make a list of positions of fields with one or more free Lorentz indices. positions of fields with more than
    #one lorentz index occur in the list as many times as there are indicies.
    field_list = spinor_contracted_term.get_field_list()
    spinor_contractions = spinor_contracted_term.get_spinor_contractions()
    lorentz_field_positions = [] #collect positions of fields associated with each lorentz index in spinor_contracted_term
    for i in range(len(field_list)):
        lorentz_rank = field_list[i].get_lorentz_rank()
        lorentz_field_positions += [i for j in range(lorentz_rank)]
    
    #generate all groupings of lorentz indices into pairs
    lorentz_contractions_list = generate_pair_partitions(lorentz_field_positions)
    
    #create new term for each item in lorentz_contractions; assemble terms into a list
    fully_contracted_list = []
    for lorentz_contractions in lorentz_contractions_list:
        new_term = term(field_list, lorentz_contractions, spinor_contractions)
        fully_contracted_list.append(new_term)
    
    return fully_contracted_list

In [42]:
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D', 'D','P']
bilinear_contraction_list = [((1, 6), '_V_'), ((2, 8), '_V_'), ((3, 10), '_T_')]
spinor_contracted_term = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
term_list = lorentz_contractions_from_spinor_contraction(spinor_contracted_term)

#get field positions of lorentz indices
field_list = term_list[0].get_field_list()
lorentz_field_positions = []
for i in range(len(field_list)):
    lorentz_rank = field_list[i].get_lorentz_rank()
    lorentz_field_positions += [i for j in range(lorentz_rank)]
    
for item in lorentz_contractions_from_spinor_contraction(spinor_contracted_term):
    print('')
    print(item.get_field_symbols())
    print(item.get_spinor_contractions())
    print(item.get_lorentz_contractions())
    print(lorentz_field_positions)
    
    
    


['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 7), (8, 10), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 7), (8, 12), (10, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 7), (8, 13), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 8), (7, 10), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), 

[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 13), (4, 7), (6, 10), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 13), (4, 7), (6, 12), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 13), (4, 8), (6, 7), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 13), (4, 8), (6, 10), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 

[(0, 13), (2, 4), (6, 10), (6, 8), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 10), (6, 12), (7, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 12), (6, 7), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 12), (6, 8), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 12), (6, 10), (7, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', '

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 4), (6, 12), (7, 13), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 4), (6, 13), (7, 8), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 4), (6, 13), (7, 10), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 6), (2, 4), (6, 13), (7, 12), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (

[(0, 13), (2, 4), (6, 7), (6, 8), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 7), (6, 10), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 7), (6, 12), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 8), (6, 7), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (2, 4), (6, 8), (6, 10), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', '

...


['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 4), (6, 12), (8, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 4), (6, 13), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 6), (4, 8), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 6), (4, 12), (8, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 


['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (0, 6), (4, 10), (6, 8), (7, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (0, 6), (4, 10), (6, 13), (7, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (0, 6), (4, 13), (6, 7), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (0, 6), (4, 13), (6, 8), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 

[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 8), (4, 13), (6, 6)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 13), (4, 6), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 13), (4, 6), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 13), (4, 8), (6, 6)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), 

[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 6), (2, 8), (6, 7), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 6), (2, 8), (6, 10), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 6), (2, 8), (6, 12), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 6), (2, 10), (6, 7), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 6), (2, 10), (6, 8), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]



[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 8), (0, 2), (4, 6), (6, 12), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 8), (0, 2), (4, 6), (6, 7), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 8), (0, 2), (4, 6), (6, 10), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 8), (0, 2), (4, 6), (6, 12), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), 

In [121]:
derivative_assignment = ['F','F','F']
bilinear_contraction_list = []
spinor_contracted_term = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
term_list = lorentz_contractions_from_spinor_contraction(spinor_contracted_term)

#get field positions of lorentz indices
field_list = term_list[0].get_field_list()
lorentz_field_positions = []
for i in range(len(field_list)):
    lorentz_rank = field_list[i].get_lorentz_rank()
    lorentz_field_positions += [i for j in range(lorentz_rank)]
    
for item in lorentz_contractions_from_spinor_contraction(spinor_contracted_term):
    print('')
    print(item.get_field_symbols())
    print(item.get_spinor_contractions())
    print(item.get_lorentz_contractions())
    print(lorentz_field_positions)
    
#create dictionary to feed into remove_zero_lorentz_contractions
lorentz_contracted_term_dict = {}
field_symbol_list = spinor_contracted_term.get_field_symbols()
spinor_contractions = spinor_contracted_term.get_spinor_contractions()
lorentz_contracted_term_dict[(tuple(field_symbol_list), tuple(spinor_contractions))]= lorentz_contractions_from_spinor_contraction(spinor_contracted_term)
print('')
print('check term dict')
for key in lorentz_contracted_term_dict.keys():
    print('key: ' + str(key))
    for item in lorentz_contracted_term_dict[key]:
        print(item)


['F', 'F', 'F']
[]
[(0, 0), (1, 1), (2, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 0), (1, 2), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 0), (1, 2), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 1), (0, 1), (2, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 1), (0, 2), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 1), (0, 2), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 1), (0, 1), (2, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 1), (0, 2), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 1), (0, 2), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 2), (0, 1), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 2), (0, 1), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 2), (0, 2), (1, 1)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 2), (0, 1), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 2), (0, 1), (1, 2)]
[0, 0, 1, 1, 2, 2]

['F', 'F', 'F']
[]
[(0, 2), (0, 2), (1, 1)]
[0, 0, 1, 1, 2, 2]

check term dict
key: (('F', 'F', 'F'), 

### Remove Redundant Lorentz Contractions

In [124]:
'''
def remove_zero_lorentz_contractions(term_dict):
    #INPUT: 
    #term_dict: dictionary with keys (derivative_assignment, bilinear_spinor_contractions) and values equal to term objects
    #OUTPUT:
    #term_dict_pruned: dictionary with relevant terms (see explanation below) removed.
    #EXPLANATION: remove terms where two lorentz indices associated with the same anti-symmetric field (i.e., F, _T_)
    #are contracted with each other, since these terms are zero (the trace of an anti-symmetric tensor is zero). 
    for term_list in term_dict.values():
        #print("len(term_list):" + str(len(term_list)))
        print('term_list: ' + str(term_list))
        for term_object in term_list:
            print(term_object)
            for contractions in term_object.get_lorentz_contractions():
                #print('-----')
                print(contractions)
                if contractions[0]==contractions[1]:
                    print('REMOVE')
                    print('-----')
                    term_list.remove(term_object)
                    break
    return term_dict
        
#lorentz_contracted_term_dict[(derivative_assignment, bilinear_spinor_contractions)]
'''

def remove_zero_lorentz_contractions(term_dict):
    #INPUT: 
    #term_dict: dictionary with keys (derivative_assignment, bilinear_spinor_contractions) and values equal to term objects
    #OUTPUT:
    #term_dict_pruned: dictionary with relevant terms (see explanation below) removed.
    #EXPLANATION: remove terms where two lorentz indices associated with the same anti-symmetric field (i.e., F, _T_)
    #are contracted with each other, since these terms are zero (the trace of an anti-symmetric tensor is zero). 
    term_dict_pruned = {}
    for key in term_dict.keys():
        term_list = term_dict[key]
        term_list_new = []
        for i in range(len(term_list)):
            term_object = term_list[i]
            if any([contraction[0]==contraction[1] for contraction in term_object.get_lorentz_contractions()]):
                continue
            term_list_new.append(term_object)
        term_dict_pruned[key] = term_list_new
    return term_dict_pruned


In [125]:
lorentz_contracted_term_dict_new = remove_zero_lorentz_contractions(lorentz_contracted_term_dict)


for key in lorentz_contracted_term_dict_new.keys():
    print('key: ' + str(key))
    for item in lorentz_contracted_term_dict_new[key]:
        print(item)


key: (('F', 'F', 'F'), ())
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 1), (0, 2), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 1), (0, 2), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 1), (0, 2), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 1), (0, 2), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 2), (0, 1), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 2), (0, 1), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 2), (0, 1), (1, 2)]
spinor contractions: []
fields: ['F', 'F', 'F']
lorentz contractions: [(0, 2), (0, 1), (1, 2)]
spinor contractions: []


In [126]:
#test remove_zero_lorentz_contractions(term_dict)

#create term_dict for input into function. do this first by generating term list, then converting term list to term
#dict
derivative_assignment = ['F','F','F']
bilinear_contraction_list = []
spinor_contracted_term = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
term_list = lorentz_contractions_from_spinor_contraction(spinor_contracted_term)

term_dict = {}

    



In [127]:
def partition_lorentz_indices(field_symbol_list, index_list = list(range(len(field_symbol_list)))):
    #partition field object positions of lorentz indices into groups, where indices in each group are mutually
    #interchangeable. here, focus on interchangeability associated with derivatives acting on the same field. 
    field_list = convert_to_field_list(field_symbol_list)
    interchangeable_index_groups = [] #list of lists, with each list containing interchangeable indices
    #index_group = [] #list of positions of derivatives acting on same field
    interchangeable_symbol_groups = []
    #symbol_group = []
    for i in range(len(field_list)):
        #print(i)
        #print(field_list[i])
        #print('')
        if field_list[i].get_symbol() == 'D':
            if i==0 or field_list[i-1].get_symbol() != 'D': #if the ith symbol is the first D in a string, create new index group and append i to it
                index_group = []
                symbol_group = []
                index_group.append(index_list[i])
                symbol_group.append('D')
            if i!=0 and field_list[i-1].get_symbol() == 'D': #if the ith D is not the first in a string of Ds, append to existing list of D's
                index_group.append(index_list[i])
                symbol_group.append('D')
            if field_list[i+1].get_symbol() != 'D': #append index group to list of groups if next element is not D
                interchangeable_index_groups.append(index_group)
                interchangeable_symbol_groups.append(symbol_group)
        if field_list[i].get_symbol() != 'D' and field_list[i].get_lorentz_rank() != 0:
            index_group = [index_list[i]]
            symbol_group = [field_list[i].get_symbol()]
            #reset list of interchangeable indices when non-derivative field occurs
            interchangeable_index_groups.append(index_group)
            interchangeable_symbol_groups.append(symbol_group)
            
    return interchangeable_index_groups, interchangeable_symbol_groups



Test *partition_lorentz_indices(field_list)*.

In [128]:
field_symbol_list = ['D', 'D', 'D', 'F', 'Pb', '_V_', 'P', 'D', 'D', 'Pb', 'D', 'D', 'D', 'D', 'P']
lorentz_contractions = [(0, 5), (1, 3), (2, 7), (3, 10), (8, 11), (12, 13)]

partition = partition_lorentz_indices(field_symbol_list, index_list = [0,4,1,3,10,9,2,11,8,7,12,5,6,14,13])
print(partition)
#lorentz_contraction_to_reduced_form(field_list, lorentz_contractions)

([[0, 4, 1], [3], [9], [11, 8], [12, 5, 6, 14]], [['D', 'D', 'D'], ['F'], ['_V_'], ['D', 'D'], ['D', 'D', 'D', 'D']])


While it is possible to interchange indices associated with derivative operators acting on the same field, it is also possible to interchange groups of indices associated with different lorentz tensors. Field objects associated with a single independent lorentz tensor are connected either by the action of a derivative on a field or by spinor contractions. 

For example, consider the field list 
- ['D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P']

with spinor contractions
- [(5, 6), (6, 10), (7, 8), (8, 11)]

The index groups associated with independent lorentz tensors here are [[0,1],[2,3,4,5,6,10],[7,8,11]]. 


In [129]:
def group_lorentz_indices(field_symbol_list, spinor_contractions):
    #assumes spinor contractions are of the form (Pb_index1, bilinear_index1), (bilinear_index1, P_index1), (Pb_index2, bilinear_index2), (bilinear_index2, P_index2), ... ]
    #EXPLANATION
    #partitions lorentz indices such that indices in each group belong to independent lorentz tensors. first build
    #up derivative groups - i.e., groups of field objects of the form ['D','D',...,'D','f'] where 'f' is any field.
    #Then use spinor contractions to connect derivative groups where 'f' is 'Pb' or 'P'. finally, single out
    #fields with non-zero lorentz rank - i.e. ignore spinor fields. 
    
    #extract indices involved in spinor contractions
    spinor_contracted_indices = []
    for contraction in spinor_contractions:
        for index in contraction:
            spinor_contracted_indices.append(index)
    
    #group derivatives acting on the same field with the field they act on
    derivative_index_group_list = [] #list of lists, with each list containing interchangeable indices
    derivative_symbol_group_list = []
    for i in range(len(field_symbol_list)):
        if field_symbol_list[i] == 'D':
            if i==0 or field_symbol_list[i-1] != 'D': #if the ith D is the first in a string, create new index group and append i to it
                index_group = []
                symbol_group = []
                index_group.append(i)
                symbol_group.append('D')
            if i!=0 and field_symbol_list[i-1] == 'D': #if the ith D is not the first in a string of Ds, append to existing list of D's
                index_group.append(i)
                symbol_group.append('D')
        if field_symbol_list[i] != 'D':
            if field_symbol_list[i-1] == 'D': #if the field object preceding a field is a D, add to group and append group to list of groups 
                index_group.append(i)
                symbol_group.append(field_symbol_list[i])
                #reset list of interchangeable indices when non-derivative field occurs
                derivative_index_group_list.append(index_group)
                derivative_symbol_group_list.append(symbol_group)
            if field_symbol_list[i-1] != 'D': #if the field object preceding a field is NOT a D, the field is in its own group as far as derivative groups are concerned
                derivative_index_group_list.append([i])
                derivative_symbol_group_list.append([field_symbol_list[i]])
    #print("derivative_index_group_list: " + str(derivative_index_group_list))
    #print("derivative_symbol_group_list: " + str(derivative_symbol_group_list))
    
    extended_index_group_list = [] #for storing independent lorentz tensors, pairs
    extended_symbol_group_list = []
    G = len(derivative_index_group_list) #also equals len(derivative_symbol_group_list)
    S = len(spinor_contractions)
    #combine derivative-based groupings connected by a spinor contraction
    for i in range(G):
        index_group = derivative_index_group_list[i]
        symbol_group = derivative_symbol_group_list[i]
        if symbol_group[-1] == 'F':
            #no spinor contractions, so no possibility for joining derivative groups
            extended_index_group_list.append(index_group)
            extended_symbol_group_list.append(symbol_group)
        if symbol_group[-1] == 'Pb':
            index_group_extended = index_group.copy() #append other group
            symbol_group_extended = symbol_group.copy()
            Pb_index = index_group[-1]
            #find blinear matrix that is contracted with Pb. loop through contractions to find it.
            for s in range(S):
                contraction = spinor_contractions[s]
                if Pb_index == contraction[0]:
                    #find index and symbol of bilinear matrix
                    bilinear_index = contraction[1]
                    bilinear_symbol = field_symbol_list[bilinear_index]
                    
                    #add bilinear to index and symbol groups
                    index_group_extended += [bilinear_index]
                    symbol_group_extended += [bilinear_symbol]
                    
                    #find P index contracted with bilinear
                    contraction_next = spinor_contractions[s+1] #makes use of assumed form of spinor_contractions
                    P_index = contraction_next[1]
                    
                    #find derivative group associated with P_index
                    for j in range(G):
                        index_group_other = derivative_index_group_list[j]
                        symbol_group_other = derivative_symbol_group_list[j]
                        if index_group_other[-1] == P_index:
                            index_group_extended += index_group_other
                            symbol_group_extended += symbol_group_other
            extended_index_group_list.append(index_group_extended)
            extended_symbol_group_list.append(symbol_group_extended)
    
    return extended_index_group_list, extended_symbol_group_list 
                    

Test *group_lorentz_indices(field_symbol_list, spinor_contractions)*.

In [130]:
field_symbol_list = ['D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P']
spinor_contractions = [(5, 6), (6, 10), (7, 8), (8, 11)]

extended_index_group_list, extended_symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)
print("extended_index_group_list: " + str(extended_index_group_list))
print("extended_symbol_group_list: " + str(extended_symbol_group_list))


extended_index_group_list: [[0, 1], [2, 3, 4, 5, 6, 9, 10], [7, 8, 11]]
extended_symbol_group_list: [['D', 'F'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P']]


Second Test with repeated lorentz tensors. 

In [131]:
field_symbol_list = ['D', 'D', 'F', 'D', 'D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P', 'Pb', '_S_', 'P']
spinor_contractions = [(9, 10),(10, 14), (11, 12), (12, 15), (16, 17), (17, 18)]

extended_index_group_list, extended_symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)
print("extended_index_group_list: " + str(extended_index_group_list))
print("extended_symbol_group_list: " + str(extended_symbol_group_list))


extended_index_group_list: [[0, 1, 2], [3, 4, 5], [6, 7, 8, 9, 10, 13, 14], [11, 12, 15], [16, 17, 18]]
extended_symbol_group_list: [['D', 'D', 'F'], ['D', 'D', 'F'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P']]


In [132]:
partition = partition_lorentz_indices(field_symbol_list)
print(partition)

IndexError: list index out of range

goal now is to convert any term object to a common standard form to determine whether two terms are equivalent (they are equivalent if they are the same when converted to standard form). produce a standard ordering of independent lorentz tensors using group_lorentz_indices, and identify subgroups within each tensor of indices that are mutually interchangeable (i.e., derivatives acting on the same field). 

In [133]:
def before_same_after(lorentz_tensor1, lorentz_tensor2):
    #INPUT 
    #lorentz_tensor1: list of field_symbols making up a lorentz tensor (i.e., no free indices of other types, such
    #as spinor indices). assumed of form ['D', ..., 'D', 'Pb', 'B', 'D', ..., 'D', 'P'], where 'B' is a dirac
    #bilinear. 
    #lorentz_tensor2: same 
    #EXPLANATION
    #returns True if symbol_list1 comes before symbol_list2 in the standard ordering, False otherwise. this
    #function serves to define the standard ordering of uncontracted lorentz tensors in a term. 
    #if 
    if lorentz_tensor1 == lorentz_tensor2:
        return 'same'
    field_ordering = ['F', 'P']
    bilinear_ordering = ['_S_', '_V_', '_T_', '_Vp_', '_Sp_']
    field1 = lorentz_tensor1[-1]
    field2 = lorentz_tensor2[-1]
    if field_ordering.index(field1) < field_ordering.index(field2):
        return 'before' 
    if field_ordering.index(field1) > field_ordering.index(field2):
        return 'after'
    elif field_ordering.index(field1) == field_ordering.index(field2):
        if field1 == 'F': #field2 must also equal F
            num_derivs1 = len(lorentz_tensor1[:-1])
            num_derivs2 = len(lorentz_tensor2[:-1])
            if num_derivs1 < num_derivs2:
                return 'before'
            if num_derivs1 == num_derivs2:
                return 'same'
            if num_derivs1 > num_derivs2:
                return 'after'  
        elif field1 == 'P': #field2 must also equal P
            #first determine bilinears of two lorentz_tensors
            bilinear1 = None
            bilinear2 = None
            for item in lorentz_tensor1:
                if item != 'D' and item != 'P' and item != 'Pb':
                    bilinear1 = item
            for item in lorentz_tensor2:
                if item != 'D' and item != 'P' and item != 'Pb':
                    bilinear2 = item
            #now compare bilinears according to ordering specified in bilinear_ordering
            if bilinear_ordering.index(bilinear1) < bilinear_ordering.index(bilinear2):
                return 'before'
            if bilinear_ordering.index(bilinear1) > bilinear_ordering.index(bilinear2):
                return 'after'
            elif bilinear_ordering.index(bilinear1) == bilinear_ordering.index(bilinear2):
                #look to number of derivatives to break tie.
                #first, order by number of derivatives acting on Pb. this is equal to the position of Pb
                d_Pb1 = lorentz_tensor1.index('Pb')
                d_Pb2 = lorentz_tensor2.index('Pb')
                if d_Pb1 < d_Pb2:
                    return 'before'
                if d_Pb1 > d_Pb2:
                    return 'after'
                #if these are equal, then order by number of derivatives acting on P
                elif d_Pb1 == d_Pb2:
                    d_P1 = lorentz_tensor1.index('P') - lorentz_tensor1.index(bilinear1) - 1
                    d_P2 = lorentz_tensor2.index('P') - lorentz_tensor2.index(bilinear2) - 1
                    if d_P1 < d_P2:
                        return 'before'
                    if d_P1 > d_P2:
                        return 'after'
                    if d_Pb1 == d_Pb2:
                        return 'same'

def sort_lorentz_tensors(index_group_list, symbol_group_list):
    #INPUT 
    #index_group_list, symbol_group_list: outputs of group_lorentz_indices function
    #OUTPUT
    #sorted version of these lists, by ordering specified in before_same_after
    symbol_group_list_sorted = [symbol_group_list[0]] #initialize sorted list with first element of unsorted list
    index_group_list_sorted = [index_group_list[0]] #initialize sorted list with first element of unsorted list
    N_group = len(symbol_group_list)
    for i in range(1, N_group): #start from second element of list since first already added to sorted list
        lorentz_tensor_new = symbol_group_list[i]
        lorentz_tensor_indices_new = index_group_list[i]
        N_sorted = len(symbol_group_list_sorted)
        for j in range(N_sorted): #loop through sorted list and keep 
            lorentz_tensor = symbol_group_list_sorted[j]
            lorentz_tensor_indices = index_group_list_sorted[j]
            relative_ordering = before_same_after(lorentz_tensor_new, lorentz_tensor)
            if relative_ordering == 'before':
                symbol_group_list_sorted.insert(j, lorentz_tensor_new)
                index_group_list_sorted.insert(j, lorentz_tensor_indices_new)
                break #element i inserted into sorted list, move to i+1
            elif j == N_sorted-1:
                symbol_group_list_sorted.append(lorentz_tensor_new)
                index_group_list_sorted.append(lorentz_tensor_indices_new)
    return index_group_list_sorted, symbol_group_list_sorted


Test before_same_after

In [134]:
lorentz_tensor1 = ['D', 'D', 'Pb','_Vp_', 'D', 'D', 'P']
lorentz_tensor2 = ['D', 'D', 'Pb','_V_', 'D', 'D', 'P']

print(before_same_after(lorentz_tensor1, lorentz_tensor2))



after


Test sort_lorentz_tensors

In [135]:
index_group_list = [[0, 1], [2, 3, 4, 5, 6, 9, 10], [7, 8, 11]]
symbol_group_list = [['D', 'F'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P']]

sort_lorentz_tensors(index_group_list, symbol_group_list)

([[0, 1], [7, 8, 11], [2, 3, 4, 5, 6, 9, 10]],
 [['D', 'F'], ['Pb', '_S_', 'P'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P']])

Test sort_lorentz_tensors on output from group_lorentz_indices. 

In [136]:
index_group_list = [[0, 1, 2], [3, 4, 5], [6, 7, 8, 9, 10, 13, 14], [11, 12, 15], [16, 17, 18]]
symbol_group_list = [['D', 'D', 'F'], ['D', 'D', 'F'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P']]

sort_lorentz_tensors(index_group_list, symbol_group_list)



([[0, 1, 2], [3, 4, 5], [11, 12, 15], [16, 17, 18], [6, 7, 8, 9, 10, 13, 14]],
 [['D', 'D', 'F'],
  ['D', 'D', 'F'],
  ['Pb', '_S_', 'P'],
  ['Pb', '_S_', 'P'],
  ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P']])

Now include lorentz contractions (spinor contractions already employed in group_lorentz_indices).

In [137]:
field_symbol_list = ['D', 'D', 'F', 'D', 'D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P', 'Pb', '_S_', 'P']
spinor_contractions = [(9, 10),(10, 14), (11, 12), (12, 15), (16, 17), (17, 18)]
lorentz_contractions = [(0, 2), (1, 3), (2, 4), (5, 8), (5, 10), (6, 13), (7, 10)] 

index_group_list, symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)
print("index_group_list: " + str(index_group_list))
print("symbol_group_list: " + str(symbol_group_list))
print("")

index_group_list_sorted, symbol_group_list_sorted = sort_lorentz_tensors(index_group_list, symbol_group_list)
print("index_group_list_sorted: " + str(index_group_list_sorted))
print("symbol_group_list_sorted: " + str(symbol_group_list_sorted))
print("")



index_group_list: [[0, 1, 2], [3, 4, 5], [6, 7, 8, 9, 10, 13, 14], [11, 12, 15], [16, 17, 18]]
symbol_group_list: [['D', 'D', 'F'], ['D', 'D', 'F'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P']]

index_group_list_sorted: [[0, 1, 2], [3, 4, 5], [11, 12, 15], [16, 17, 18], [6, 7, 8, 9, 10, 13, 14]]
symbol_group_list_sorted: [['D', 'D', 'F'], ['D', 'D', 'F'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P']]



In [138]:
def partition_sublists(index_group_list, symbol_group_list):
    N_groups = len(symbol_group_list) #could also use symbol_group_list
    index_group_list_partitioned = []
    symbol_group_list_partitioned = []
    for i in range(N_groups):
        field_symbol_list = symbol_group_list[i]
        index_list = index_group_list[i]
        index_partition, symbol_partition = partition_lorentz_indices(field_symbol_list, index_list)
        index_group_list_partitioned.append(index_partition)
        symbol_group_list_partitioned.append(symbol_partition)
    return index_group_list_partitioned, symbol_group_list_partitioned

In [139]:
index_group_list_sorted_partitioned, symbol_group_list_sorted_partitioned = partition_sublists(index_group_list_sorted, symbol_group_list_sorted)
print("index_group_list_sorted_partitioned: " + str(index_group_list_sorted_partitioned))
print("symbol_group_list_sorted_partitioned: " + str(symbol_group_list_sorted_partitioned))

index_group_list_sorted_partitioned: [[[0, 1], [2]], [[3, 4], [5]], [], [], [[6, 7, 8], [10], [13]]]
symbol_group_list_sorted_partitioned: [[['D', 'D'], ['F']], [['D', 'D'], ['F']], [], [], [['D', 'D', 'D'], ['_T_'], ['D']]]


In [140]:
lorentz_contractions1 = [(0, 2), (1, 3), (2, 4), (5, 8), (5, 10), (6, 13), (7, 10)] 
reduced_contractions1 = [([0,0],[0,1]), ([0,0],[1,0]), ([0,1],[1,0]), ([1,1],[4,0]), ([1,1],[4,1]), ([4,0],[4,2]), ([4,0],[4,1])]

lorentz_contractions2 = [(3, 5), (4, 0), (5, 1), (2, 8), (2, 10), (6, 13), (7, 10)] 
#reduced_contractions2 = [([1,0],[,]), ([,],[,]), ([,],[,]), ([,],[,]), ([,],[,]), ([4,0],[4,2]), ([4,0],[4,1])]


Use Lorentz contractions to decide which group of indices, $[[0, 1], [2]]$ or $[[3, 4], [5]]$, comes first (given that both are associated with the lorentz tensor $[['D', 'D'], ['F']]$). By convention, place first the group of indices that is contracted with the earliest outside (not associated with a $[['D', 'D'], ['F']]$ tensor) group of lorentz indices. If there are no contractions with outside lorentz tensors; if a single lorentz index is left uncontracted in one, put first the one with no free indices; if no free lorentz index is in either term, put first the one in which the indices of non-derivative lorentz tensor are contracted into the smaller combined number of derivatives (add up the number of derivatives in each group into which the indices or index of the non-deriative tensor is contracted).  

In [141]:
#symbol_groups = [['D', 'D', 'F'], ['D', 'D', 'F'], [], [], ['D', 'D', 'D', '_T_', 'D']]
#symbol_groups_unique = [list(x) for x in set(tuple(x) for x in symbol_groups)]
#symbol_groups_unique

Use partition_lorentz_indices on each lorentz tensor subgroup to identify interchangeable derivative positions.

In [142]:

def reduce_lorentz_contractions(term_object):
    #print("")
    #print("In reduce_lorentz_contractions(term_object)")
    #print("type(term_object): " + str(type(term_object)))
    #print(term_object)
    field_symbol_list = term_object.get_field_symbols()
    spinor_contractions = term_object.get_spinor_contractions()
    lorentz_contractions = term_object.get_lorentz_contractions()
    #print("type(field_symbol_list): " + str(type(field_symbol_list)))
    #print(str(field_symbol_list))
    #print("type(lorentz_contractions): " + str(type(lorentz_contractions)))
    #print(lorentz_contractions)
    #print("type(spinor_contractions): " + str(type(spinor_contractions)))
    #print(spinor_contractions)
    
    #group field symbols and positions associated with different lorentz tensors
    index_group_list, symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)
    #sort field symbol list and index list of groups associated with different lorentz tensors
    index_group_list_sorted, symbol_group_list_sorted = sort_lorentz_tensors(index_group_list, symbol_group_list)
    #partition each group associated with each lorentz tensor into groups of mutually interchangable positions (associated with commutation of derivatives)
    index_group_list_sorted_partitioned, symbol_group_list_sorted_partitioned = partition_sublists(index_group_list_sorted, symbol_group_list_sorted)
    
    #convert indices of lorentz fields to reduced form 
    N_lorentz = len(lorentz_contractions)
    lorentz_contractions_reduced = []
    for i in range(N_lorentz):
        contraction_i = lorentz_contractions[i]
        first_index = contraction_i[0]
        second_index = contraction_i[1]
        #check which subgroup of which group the first_index and second_index belong to
        for j in range(len(index_group_list_sorted_partitioned)):
            group_j = index_group_list_sorted_partitioned[j]
            for k in range(len(group_j)):
                subgroup_jk = group_j[k]
                if first_index in subgroup_jk:
                    first_index_reduced = [j, k]
                if second_index in subgroup_jk:
                    second_index_reduced = [j, k]
        #fill contraction_reduced and append to lorentz_contractions_reduced
        contraction_reduced = (first_index_reduced, second_index_reduced)
        lorentz_contractions_reduced.append(contraction_reduced)
    
    return lorentz_contractions_reduced


In [143]:
field_symbol_list = ['D', 'D', 'F', 'D', 'D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P', 'Pb', '_S_', 'P']
field_list = convert_to_field_list(field_symbol_list)
spinor_contractions = [(9, 10),(10, 14), (11, 12), (12, 15), (16, 17), (17, 18)]
lorentz_contractions = [(0, 2), (1, 3), (2, 4), (5, 8), (5, 10), (6, 13), (7, 10)] 
#reduced_contractions = [([0,0],[0,1]), ([0,0],[1,0]), ([0,1],[1,0]), ([1,1],[4,0]), ([1,1],[4,1]), ([4,0],[4,2]), ([4,0],[4,1])]

index_group_list, symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)
print("index_group_list: " + str(index_group_list))
print("symbol_group_list: " + str(symbol_group_list))
print("")
index_group_list_sorted, symbol_group_list_sorted = sort_lorentz_tensors(index_group_list, symbol_group_list)
print("index_group_list_sorted: " + str(index_group_list_sorted))
print("symbol_group_list_sorted: " + str(symbol_group_list_sorted))
print("")
index_group_list_sorted_partitioned, symbol_group_list_sorted_partitioned = partition_sublists(index_group_list_sorted, symbol_group_list_sorted)
print("index_group_list_sorted_partitioned: " + str(index_group_list_sorted_partitioned))
print("symbol_group_list_sorted_partitioned: " + str(symbol_group_list_sorted_partitioned))

term_object = term(field_list, lorentz_contractions, spinor_contractions)
lorentz_contractions_reduced = reduce_lorentz_contractions(term_object)
print("lorentz_contractions_reduced: " + str(lorentz_contractions_reduced))

lorentz_contractions2 = [(3, 5), (4, 0), (5, 1), (2, 8), (2, 10), (6, 13), (7, 10)] 
term_object2 = term(field_list, lorentz_contractions2, spinor_contractions)
lorentz_contractions_reduced2 = reduce_lorentz_contractions(term_object2)
print("lorentz_contractions_reduced2: " + str(lorentz_contractions_reduced2))




index_group_list: [[0, 1, 2], [3, 4, 5], [6, 7, 8, 9, 10, 13, 14], [11, 12, 15], [16, 17, 18]]
symbol_group_list: [['D', 'D', 'F'], ['D', 'D', 'F'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P']]

index_group_list_sorted: [[0, 1, 2], [3, 4, 5], [11, 12, 15], [16, 17, 18], [6, 7, 8, 9, 10, 13, 14]]
symbol_group_list_sorted: [['D', 'D', 'F'], ['D', 'D', 'F'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P']]

index_group_list_sorted_partitioned: [[[0, 1], [2]], [[3, 4], [5]], [], [], [[6, 7, 8], [10], [13]]]
symbol_group_list_sorted_partitioned: [[['D', 'D'], ['F']], [['D', 'D'], ['F']], [], [], [['D', 'D', 'D'], ['_T_'], ['D']]]
lorentz_contractions_reduced: [([0, 0], [0, 1]), ([0, 0], [1, 0]), ([0, 1], [1, 0]), ([1, 1], [4, 0]), ([1, 1], [4, 1]), ([4, 0], [4, 2]), ([4, 0], [4, 1])]
lorentz_contractions_reduced2: [([1, 0], [1, 1]), ([1, 0], [0, 0]), ([1, 1], [0, 0]), ([0, 1], [4, 0]), ([0, 1], [4, 1]), ([4, 0], [4, 2]), ([4,

- first, sort reduced contractions by bin and by subgroup order (do not sort by group within bin)
- second, for each bin, list group numbers in order of occurrence in the sorted list of contractions.
- sort list of group numbers for each bin. 
- replace each group number in original list of contractions with its corresponding element in the sorted list. 

In [144]:
def sort_reduced_lorentz_contractions(lorentz_contractions_reduced, symbol_group_list_sorted):
    #bin together group indices that belong to the same lorentz tensor. 
    if lorentz_contractions_reduced == []:
        return []
    else:
        binned_group_indices = []
        for i in range(len(symbol_group_list_sorted)):
            if i==0 or symbol_group_list_sorted[i]!=symbol_group_list_sorted[i-1]:
                binned_group_indices.append([i])
            elif symbol_group_list_sorted[i]==symbol_group_list_sorted[i-1]:
                last_bin = binned_group_indices[-1]
                last_bin.append(i)
        #print("binned_group_indices: " + str(binned_group_indices))

        lorentz_contractions_reduced_sorted = [lorentz_contractions_reduced[0]] #initialize with first element of lorentz_contractions_reduced
        for i in range(1, len(lorentz_contractions_reduced)):
            reduced_contraction = lorentz_contractions_reduced[i]
            for j in range(len(lorentz_contractions_reduced_sorted)):
                reduced_contraction_sorted = lorentz_contractions_reduced_sorted[j]
                relative_order = compare_reduced_contractions(reduced_contraction, reduced_contraction_sorted, symbol_group_list_sorted)
                if relative_order == "before" or relative_order == "same":
                    lorentz_contractions_reduced_sorted.insert(j, reduced_contraction)
                    break
                elif j == len(lorentz_contractions_reduced_sorted) - 1:
                    lorentz_contractions_reduced_sorted.append(reduced_contraction)

        return lorentz_contractions_reduced_sorted

def compare_reduced_contractions(reduced_contraction1, reduced_contraction2, symbol_group_list_sorted): 
    #compare reduced contractions in order by 1) bin of first index, 2) subgroup of first index, 
    #3) bin of second index, 4) subgroup of second index
    binned_group_indices = []
    for i in range(len(symbol_group_list_sorted)):
        if i==0 or symbol_group_list_sorted[i]!=symbol_group_list_sorted[i-1]:
            binned_group_indices.append([i])
        elif symbol_group_list_sorted[i]==symbol_group_list_sorted[i-1]:
            last_bin = binned_group_indices[-1]
            last_bin.append(i)
    ##print("binned_group_indices: " + str(binned_group_indices))
    
    first_index1 = reduced_contraction1[0]
    second_index1 = reduced_contraction1[1]
    first_index_group1 = first_index1[0]
    first_index_subgroup1 = first_index1[1]
    second_index_group1 = second_index1[0]
    second_index_subgroup1 = second_index1[1]
    
    first_index2 = reduced_contraction2[0]
    second_index2 = reduced_contraction2[1]   
    first_index_group2 = first_index2[0]
    first_index_subgroup2 = first_index2[1]
    second_index_group2 = second_index2[0]
    second_index_subgroup2 = second_index2[1]
    
    #initialize binned version of contraction 1
    first_index_binned1 = [None, first_index_subgroup1] #initialize binned contraction index
    second_index_binned1 = [None, second_index_subgroup1] #initialize binned contraction index
    #initialized binned version of contraction 2
    first_index_binned2 = [None, first_index_subgroup2] #initialize binned contraction index
    second_index_binned2 = [None, second_index_subgroup2] #initialize binned contraction index
    for j in range(len(binned_group_indices)):
        group_bin = binned_group_indices[j]
        #bin groups in first contraction
        if first_index_group1 in group_bin:
            first_index_binned1[0] = j
        if second_index_group1 in group_bin:
            second_index_binned1[0] = j
        #bin groups in second contraction
        if first_index_group2 in group_bin:
            first_index_binned2[0] = j
        if second_index_group2 in group_bin:
            second_index_binned2[0] = j            
    reduced_contraction_binned1 = [first_index_binned1, second_index_binned1]
    reduced_contraction_binned2 = [first_index_binned2, second_index_binned2]
    
    #groups and subgroups for first contraction
    first_index_bin1 = first_index_binned1[0]
    first_index_subgroup1 = first_index_binned1[1]
    second_index_bin1 = second_index_binned1[0]
    second_index_subgroup1 = second_index_binned1[1]
    
    #groups and subgroups for second contraction
    first_index_bin2 = first_index_binned2[0]
    first_index_subgroup2 = first_index_binned2[1]
    second_index_bin2 = second_index_binned2[0]
    second_index_subgroup2 = second_index_binned2[1]
    
    if first_index_bin1 < first_index_bin2:
        return "before"
    elif first_index_bin1 < first_index_bin2:
        return "after"
    elif first_index_bin1 == first_index_bin2:
        if first_index_subgroup1 < first_index_subgroup2:
            return "before"
        elif first_index_subgroup1 > first_index_subgroup2:
            return "after"
        elif first_index_subgroup1 == first_index_subgroup2:
            if second_index_bin1 < second_index_bin2:
                return "before"
            elif second_index_bin1 < second_index_bin2:
                return "after"
            elif second_index_bin1 == second_index_bin2:
                if second_index_subgroup1 < second_index_subgroup2:
                    return "before"
                elif second_index_subgroup1 > second_index_subgroup2:
                    return "after"
                elif second_index_subgroup1 == second_index_subgroup2:
                    return "same"
            
       
    

In [145]:
symbol_group_list_sorted = [['D', 'D', 'F'], ['D', 'D', 'F'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P']]
lorentz_contractions_reduced = [([0, 0], [0, 1]), ([0, 0], [1, 0]), ([0, 1], [1, 0]), ([1, 1], [4, 0]), ([1, 1], [4, 1]), ([4, 0], [4, 2]), ([4, 0], [4, 1])]
lorentz_contractions_reduced2 = [([1, 0], [1, 1]), ([1, 0], [0, 0]), ([1, 1], [0, 0]), ([0, 1], [4, 0]), ([0, 1], [4, 1]), ([4, 0], [4, 2]), ([4, 0], [4, 1])]
#lorentz_contractions_reduced2 = [([1, 0], [1, 1]), ([1, 0], [0, 0]), ([1, 1], [0, 0]), ([0, 1], [4, 0]), ([0, 1], [4, 1]), ([4, 0], [4, 2]), ([4, 0], [4, 1])]
print(sort_reduced_lorentz_contractions(lorentz_contractions_reduced, symbol_group_list_sorted))
print(sort_reduced_lorentz_contractions(lorentz_contractions_reduced2, symbol_group_list_sorted))

#reduced_lorentz_contractions_equiv(lorentz_contractions_reduced1, lorentz_contractions_reduced2, symbol_group_list_sorted)

[([0, 0], [1, 0]), ([0, 0], [0, 1]), ([0, 1], [1, 0]), ([1, 1], [4, 0]), ([1, 1], [4, 1]), ([4, 0], [4, 1]), ([4, 0], [4, 2])]
[([1, 0], [0, 0]), ([1, 0], [1, 1]), ([1, 1], [0, 0]), ([0, 1], [4, 0]), ([0, 1], [4, 1]), ([4, 0], [4, 1]), ([4, 0], [4, 2])]


In [146]:

def sort_within_bins(reduced_contractions_sorted, symbol_group_list_sorted): 
    #EXPLANATION: for repeated lorentz tensors in reduced_contractions_sorted symbol_group_list_sorted, put
    #contractions into a standard form. 
    
    #print("reduced_contractions_sorted: " + str(reduced_contractions_sorted))
    #print("symbol_group_list_sorted: " + str(symbol_group_list_sorted))
    
    #bin together group indices that belong to the same lorentz tensor. this only uses symbol_group_list_sorted
    binned_group_indices = []
    for i in range(len(symbol_group_list_sorted)):
        if i==0 or symbol_group_list_sorted[i]!=symbol_group_list_sorted[i-1]:
            binned_group_indices.append([i]) #start new bin if symbol group list different from previous
        elif symbol_group_list_sorted[i]==symbol_group_list_sorted[i-1]:
            last_bin = binned_group_indices[-1]
            last_bin.append(i)
    #print("binned_group_indices: " + str(binned_group_indices))
    N_bins = len(binned_group_indices)       
    #initialize reduced contraction list. loop through each bin and replace group indices with their sorted values.
    #reduced_contractions_bin_sorted = []
    #for each bin, 
    #I) assemble group (as opposed to subgroup) parts of contracted indices from that bin in order of first appearance within reduced_contractions_sorted.
    #II) after sorting this list, replace each contracted group index with its counterpart in the sorted list  
    
    #I)
    group_indices_unique_list = []
    group_indices_sorted_list = []
    for group_bin in binned_group_indices:
        #I) 
        #print("In sort_within_bins")
        #print("group_bin: " + str(group_bin))
        group_indices = [] #list in order of appearance in list of contractions all group components from group_bin
        for contraction in reduced_contractions_sorted:
            reduced_index1 = contraction[0] #first reduced index in contraction
            #print("reduced_index1: " + str(reduced_index1))
            reduced_index2 = contraction[1] #second reduced index in contraction
            #print("reduced_index2: " + str(reduced_index2))
            if reduced_index1[0] in group_bin:
                group_indices.append(reduced_index1[0])
            if reduced_index2[0] in group_bin:
                group_indices.append(reduced_index2[0])
        group_indices_unique = unique(group_indices)
        group_indices_sorted = sorted(group_indices_unique)
        #print("group_indices_unique: " + str(group_indices_unique))
        #print("group_indices_sorted: " + str(group_indices_sorted))
        group_indices_unique_list.append(group_indices_unique)
        group_indices_sorted_list.append(group_indices_sorted)
        
    #II)
    reduced_contractions_bin_sorted = []
    for i in range(len(reduced_contractions_sorted)): #for each contraction, replace it with contraction_new
        contraction_old = list(reduced_contractions_sorted[i])
        
        reduced_index_old1 = contraction_old[0]
        reduced_index_old2 = contraction_old[1]
        
        group_index_old1 = reduced_index_old1[0]
        subgroup_index_old1 = reduced_index_old1[1]
        group_index_old2 = reduced_index_old2[0]
        subgroup_index_old2 = reduced_index_old2[1]
        
        contraction_new = [[group_index_old1,subgroup_index_old1], [group_index_old2,subgroup_index_old2]] #initialize new reduced contraction
        for j in range(N_bins): #loop through bins; for whenever one of the contraction group indices is equl
            #to one of the group indices in a bin - i.e., is in group_indices_unique - replace that group index
            #with the corresponding index in group_indices_sorted.
            group_indices_unique = group_indices_unique_list[j]
            group_indices_sorted = group_indices_sorted_list[j]
            N_unique = len(group_indices_unique)
            for k in range(N_unique):
                if reduced_contractions_sorted[i][0][0] == group_indices_unique[k]:
                    contraction_new[0][0] = group_indices_sorted[k]
                if reduced_contractions_sorted[i][1][0] == group_indices_unique[k]:
                    contraction_new[1][0] = group_indices_sorted[k]
        #print("contraction_new: " + str(contraction_new))
        reduced_contractions_bin_sorted.append(contraction_new)
            
    return reduced_contractions_bin_sorted

def unique(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
            

In [147]:
reduced_contractions_sorted = [([0, 0], [1, 0])]
symbol_group_list_sorted = [['Pb', '_V_', 'P'], ['Pb', '_T_', 'P']]
sort_within_bins(reduced_contractions_sorted, symbol_group_list_sorted)

[[[0, 0], [1, 0]]]

In [148]:
symbol_group_list_sorted = [['D', 'D', 'F'], ['D', 'D', 'F'], ['Pb', '_S_', 'P'], ['Pb', '_S_', 'P'], ['D', 'D', 'D', 'Pb', '_T_', 'D', 'P']]
reduced_contractions_sorted1 = [([0, 0], [1, 0]), ([0, 0], [0, 1]), ([0, 1], [1, 0]), ([1, 1], [4, 0]), ([1, 1], [4, 1]), ([4, 0], [4, 1]), ([4, 0], [4, 2])]
reduced_contractions_sorted2 = [([1, 0], [0, 0]), ([1, 0], [1, 1]), ([1, 1], [0, 0]), ([0, 1], [4, 0]), ([0, 1], [4, 1]), ([4, 0], [4, 1]), ([4, 0], [4, 2])]
reduced_contractions_sorted_bins1 = sort_within_bins(reduced_contractions_sorted1, symbol_group_list_sorted)
print("")
print("")
reduced_contractions_sorted_bins2 = sort_within_bins(reduced_contractions_sorted2, symbol_group_list_sorted)
print("reduced_contractions_sorted_bins1: " + str(reduced_contractions_sorted_bins1))
print("reduced_contractions_sorted_bins2: " + str(reduced_contractions_sorted_bins2))



reduced_contractions_sorted_bins1: [[[0, 0], [1, 0]], [[0, 0], [0, 1]], [[0, 1], [1, 0]], [[1, 1], [4, 0]], [[1, 1], [4, 1]], [[4, 0], [4, 1]], [[4, 0], [4, 2]]]
reduced_contractions_sorted_bins2: [[[0, 0], [1, 0]], [[0, 0], [0, 1]], [[0, 1], [1, 0]], [[1, 1], [4, 0]], [[1, 1], [4, 1]], [[4, 0], [4, 1]], [[4, 0], [4, 2]]]


Now, time to put the above pieces together to generate inequivalent terms (prior to imposing Fierz, IBP, and EOM). Let's review the steps taken and the functions that perform them:
- generate all combinations of fields with a given mass dimension: *generate_field_combos(massDim)*
- for each such combination of fields, generate all possible derivative assignments: *generate_derivative_assignments(field_combo)*
- for each derivative assignment, generate all inquivalent spinor contractions for all bilinears: *reduced_spinor_contractions(derivative_assignment)*
- for each derivative assignment and spinor contraction, generate a term object with empty lorentz indices: *convert_to_term_object(derivative_assignment, bilinear_contraction_list)*
- for each term object specified by a particular derivative assignment and set of spinor contractions, generate all possible ways of contracting terms with non-zero lorentz rank: *lorentz_contractions_from_spinor_contraction(spinor_contracted_term)*
- eliminate redundant lorentz contractions, where the redundancy is associated specifically with commutation of derivatives or of multiple factors of the same lorentz tensor, or with terms that are zero because contracted indices are both on the same antisymmetric lorentz tensor. 
    - first, eliminate zero terms associated with self-contracted antisymmetric lorentz tensors


In [149]:
def reduced_contractions_equiv(reduced_contractions1, reduced_contractions2):
    return reduced_contractions1==reduced_contractions2

reduced_contractions1 = [([1, 0], [1, 0]), ([1, 0], [1, 1]), ([1, 1], [1, 0]), ([1, 1], [4, 0]), ([1, 1], [4, 1]), ([4, 0], [4, 1]), ([4, 0], [4, 2])]
reduced_contractions2 = [([1, 0], [1, 0]), ([1, 0], [1, 1]), ([1, 1], [1, 0]), ([1, 1], [4, 0]), ([1, 1], [4, 1]), ([4, 0], [4, 1]), ([4, 0], [4, 2])]

reduced_contractions_equiv(reduced_contractions1, reduced_contractions2)

True

In [150]:
def lorentz_contractions_equiv(lorentz_contractions1, lorentz_contractions2, field_symbol_list, spinor_contractions):
    #EXPLANATION: determine whether two lorentz contractions are equivalent relative to a particular field list and set of spinor contractions. 
    
    #convert field symbol list to field list and define terms with different lorentz contractions for input into 
    #reduce_lorentz_contractions() function. 
    field_list = convert_to_field_list(field_symbol_list)
    term1 = term(field_list, lorentz_contractions1, spinor_contractions)
    term2 = term(field_list, lorentz_contractions2, spinor_contractions)
    
    #find reduced lorentz contractions (makes use of spinor contractions)
    reduced_lorentz_contractions1 = reduce_lorentz_contractions(term1)
    reduced_lorentz_contractions2 = reduce_lorentz_contractions(term2)
    
    #group field symbol list into independent lorentz tensors
    index_group_list, symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)
    
    #sort independent lorentz tensors for comparison of lorentz contractions
    _, symbol_group_list_sorted = sort_lorentz_tensors(index_group_list, symbol_group_list)
    
    #sort reduced contractions by bin and subgroup
    lorentz_contractions_reduced_groupSorted1 = sort_reduced_lorentz_contractions(reduced_lorentz_contractions1, symbol_group_list_sorted)
    lorentz_contractions_reduced_groupSorted2 = sort_reduced_lorentz_contractions(reduced_lorentz_contractions2, symbol_group_list_sorted)

    #then sort reduced contractions within bin
    lorentz_contractions_reduced_groupSorted_binSorted1 = sort_within_bins(lorentz_contractions_reduced_groupSorted1, symbol_group_list_sorted)
    lorentz_contractions_reduced_groupSorted_binSorted2 = sort_within_bins(lorentz_contractions_reduced_groupSorted2, symbol_group_list_sorted)
    
    #terms are equivalent if reduced and fully sorted lorentz contractions are equal (assuming spinor contractions and fields lists are equivalent)
    equiv = lorentz_contractions_reduced_groupSorted_binSorted1==lorentz_contractions_reduced_groupSorted_binSorted2
    
    return equiv
    

Test *lorentz_contractions_equiv(lorentz_contractions1, lorentz_contractions2, field_symbol_list, spinor_contractions)*.

In [151]:
#set test inputs
field_symbol_list = ['D', 'D', 'F', 'D', 'D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P', 'Pb', '_S_', 'P']
spinor_contractions = [(9, 10),(10, 14), (11, 12), (12, 15), (16, 17), (17, 18)]
lorentz_contractions1 = [(0, 2), (1, 3), (2, 4), (5, 8), (5, 10), (6, 13), (7, 10)]
lorentz_contractions2 = [(3, 5), (4, 0), (5, 1), (2, 8), (2, 10), (6, 13), (7, 10)] 

equiv = lorentz_contractions_equiv(lorentz_contractions1, lorentz_contractions2, field_symbol_list, spinor_contractions)
print(equiv)

True


Second test of *lorentz_contractions_equiv(lorentz_contractions1, lorentz_contractions2, field_symbol_list, spinor_contractions)*. This time, allow field symbol list to be more unsorted. 

In [152]:
field_symbol_list = ['D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'D', 'D', 'F', 'D', 'D', 'Pb', '_T_', 'D', 'D', 'F', 'D', 'P', 'P']
spinor_contractions = [(2, 3), (3, 7), (4, 5), (5, 20), (13, 14), (14, 19)]
index_group_list, symbol_group_list = group_lorentz_indices(field_symbol_list, spinor_contractions)

print(index_group_list)
print(symbol_group_list)
'''
lorentz tensor groups: ([[0, 1, 2, 3, 6, 7],
  [4, 5, 20],
  [8, 9, 10],
  [11, 12, 13, 14, 18, 19],
  [15, 16, 17]],
 [['D', 'D', 'Pb', '_T_', 'D', 'P'],
  ['Pb', 'S', 'P'],
  ['D', 'D', 'F'],
  ['D', 'D', 'Pb', '_T_', 'D', 'P'],
  ['D', 'D', 'F']])
'''

#interchange [0, 1, 2, 3, 6, 7] group with [11, 12, 13, 14, 18, 19] group
lorentz_contractions1 = [(0, 3), (1, 6), (3, 10), (8, 9), (10, 17), (11, 18), (12, 14), (14, 17), (15, 16)] 
#lorentz_contractions2 = [(12, 14), (11, 18), (14, 10), (9, 8), (10, 17), (0, 6), (1, 3), (3, 17), (15, 16)] 
lorentz_contractions2 = [(12, 14), (11, 18), (14, 17), (16, 15), (17, 10), (0, 6), (1, 3), (3, 10), (8, 9)] 
#lorentz_contractions2 = [(11, 3), (12, 6), (14, 10), (8, 9), (10, 17), (0, 6), (1, 3), (6, 17), (15, 16)]
equiv = lorentz_contractions_equiv(lorentz_contractions1, lorentz_contractions2, field_symbol_list, spinor_contractions)
print(equiv)

#field_list = convert_to_field_list(field_symbol_list)
#term1 = term(field_list, lorentz_contractions, spinor_contractions)
#index_group_list_sorted, symbol_group_list_sorted = sort_lorentz_tensors(index_group_list, symbol_group_list)


[[0, 1, 2, 3, 6, 7], [4, 5, 20], [8, 9, 10], [11, 12, 13, 14, 18, 19], [15, 16, 17]]
[['D', 'D', 'Pb', '_T_', 'D', 'P'], ['Pb', '_S_', 'P'], ['D', 'D', 'F'], ['D', 'D', 'Pb', '_T_', 'D', 'P'], ['D', 'D', 'F']]
True


In [153]:
def generate(massDim):
    #EXPLANATION
    #generates all inequivalent terms of given mass dimension.
    #generate field combinations
    print("Generating field combinations...")
    field_combo_symbol_list = generate_field_combos_reduced(massDim) #lists of field objects
    print(field_combo_symbol_list)
    #field_combo_symbol_list = []
    #for field_combo in field_combo_list:
        #field_combo = convert_to_symbol_list(field_combo)
        #field_combo_symbol_list.append(field_combo)
    #print(field_combo_symbol_list)
        
    #generate list of all derivative assignments
    print("Generating list of derivative assignments...")
    derivative_assignments_list = []
    for field_combo in field_combo_symbol_list:
        #field_combo_list = convert_to_symbol_list(field_combo_list)
        print(generate_derivative_assignments(field_combo))
        derivative_assignments_list += generate_derivative_assignments(field_combo)
    #print(derivative_assignments_list)
    
    #generate all inequivalent spinor contractions for each derivative assignment
    print("Generating (derivative assignments, spinor contractions) pairs...")
    bilinear_spinor_contractions_dict = {}
    for derivative_assignment in derivative_assignments_list:
        bilinear_spinor_contractions_dict[tuple(derivative_assignment)] = reduced_spinor_contractions(derivative_assignment)
    print("bilinear_spinor_contractions_dict: " + str(bilinear_spinor_contractions_dict))
    
    #convert (derivative_assignment, spinor_contraction) pairs into term objects
    print("Converting (derivative assignments, spinor contractions) pairs into term objects...")
    spinor_contracted_term_dict = {}
    for derivative_assignment in bilinear_spinor_contractions_dict.keys():
        print("DERIVATIVE ASSIGNMENT: " + str(derivative_assignment))
        if not bilinear_spinor_contractions_dict[derivative_assignment]:
            new_term = convert_to_term_object(list(derivative_assignment), [])
            spinor_contracted_term_dict[(tuple(derivative_assignment), tuple([]))] = new_term
        else:
            for bilinear_spinor_contractions in bilinear_spinor_contractions_dict[derivative_assignment]:
                print("BILINEAR SPINOR CONTRACTIONs: " + str(bilinear_spinor_contractions))
                new_term = convert_to_term_object(list(derivative_assignment), bilinear_spinor_contractions)
                spinor_contracted_term_dict[(tuple(derivative_assignment), tuple(bilinear_spinor_contractions))] = new_term
    print('')
    print("spinor_contracted_term_dict: ")
    for item in spinor_contracted_term_dict.values():
        print(item)
    
    #generate all lorentz contractions from derivative assignment and spinor contraction 
    print("Generating lorentz contractions...")
    lorentz_contracted_term_dict = {}
    for key in spinor_contracted_term_dict.keys():
        spinor_contracted_term = spinor_contracted_term_dict[key]
        field_symbol_list = spinor_contracted_term.get_field_symbols()
        spinor_contractions = spinor_contracted_term.get_spinor_contractions()
        lorentz_contracted_term_dict[(tuple(field_symbol_list), tuple(spinor_contractions))]= lorentz_contractions_from_spinor_contraction(spinor_contracted_term)
    #remove lorentz contractions on same field
    #lorentz_contracted_term_dict_zeros_removed = {}
    #term_list_pruned = remove_zero_lorentz_contractions(term_list)
    
    #remove zero lorentz contractions
    print("Removing terms with self contractions on anti-symmetric fields...")
    lorentz_contracted_term_dict = remove_zero_lorentz_contractions(lorentz_contracted_term_dict)
    
    #remove repeat lorentz contractions
    print("Removing repeat lorentz contractions...")
    lorentz_contracted_term_dict_unique = {}
    for key in lorentz_contracted_term_dict.keys():
        #print("key: " + str(key))
        #print("lorentz_contracted_term_dict[key]: ")
        #[print(str(i)) for i in lorentz_contracted_term_dict[key]]
        field_symbol_list = key[0]
        spinor_contractions = key[1]
        lorentz_contractions_lists = lorentz_contracted_term_dict[key]
        lorentz_contractions_lists_unique = [] #initialize with
        for term1 in lorentz_contractions_lists:
            #print("")
            #print("term1: " + str(term1))
            lorentz_contractions1 = term1.get_lorentz_contractions()
            repeat = False #append term1 to unique list if repeat=False. initialize to False
            for term2 in lorentz_contractions_lists_unique:
                #print("term2: " + str(term2))
                lorentz_contractions2 = term2.get_lorentz_contractions()
                if lorentz_contractions_equiv(lorentz_contractions1, lorentz_contractions2, field_symbol_list, spinor_contractions):
                    repeat = True
                    break 
            if not repeat:
                lorentz_contractions_lists_unique.append(term1)
            #if term1 not in lorentz_contractions_lists_unique:
                #lorentz_contractions_lists_unique.append(term1)
        lorentz_contracted_term_dict_unique[key] = lorentz_contractions_lists_unique
    
    #convert lorentz_contracted_term_dict_unique to term list
    print("Converting to term list ...")
    term_list_unique = []
    for term_list in lorentz_contracted_term_dict_unique.values():
        for term_object in term_list:
             term_list_unique.append(term_object)
            
    return term_list_unique
    
    

Test generate(massDim).

In [154]:
massDim = 6
terms_unique = generate(massDim)
for term_object in terms_unique:
    print("")
    print(term_object)

Generating field combinations...
[['F', 'F', 'F'], ['Pb', 'P', 'Pb', 'P'], ['D', 'F', 'Pb', 'P'], ['D', 'D', 'F', 'F'], ['D', 'D', 'D', 'Pb', 'P'], ['D', 'D', 'D', 'D', 'F']]
Generating list of derivative assignments...
[['F', 'F', 'F']]
[['Pb', 'Pb', 'P', 'P']]
[['F', 'Pb', 'D', 'P'], ['F', 'D', 'Pb', 'P'], ['D', 'F', 'Pb', 'P']]
[['D', 'F', 'D', 'F'], ['D', 'D', 'F', 'F']]
[['Pb', 'D', 'D', 'D', 'P'], ['D', 'Pb', 'D', 'D', 'P'], ['D', 'D', 'Pb', 'D', 'P'], ['D', 'D', 'D', 'Pb', 'P']]
[['D', 'D', 'D', 'D', 'F']]
Generating (derivative assignments, spinor contractions) pairs...
bilinear_spinor_contractions_dict: {('F', 'F', 'F'): [], ('Pb', 'Pb', 'P', 'P'): [[((0, 2), '_S_'), ((1, 3), '_S_')], [((0, 2), '_S_'), ((1, 3), '_V_')], [((0, 2), '_S_'), ((1, 3), '_T_')], [((0, 2), '_S_'), ((1, 3), '_Vp_')], [((0, 2), '_S_'), ((1, 3), '_Sp_')], [((0, 2), '_V_'), ((1, 3), '_V_')], [((0, 2), '_V_'), ((1, 3), '_T_')], [((0, 2), '_V_'), ((1, 3), '_Vp_')], [((0, 2), '_V_'), ((1, 3), '_Sp_')], [((0,

Missing $F_{\mu \nu}F^{\mu \nu}$ term. Go back to generation of spinor contractions, make sure this is included. Corrected.

Now, repeats of FFF term, and inclusion of term where antisymmetric F is contracted with itself (should be zero). Corrected.

Still, repeat of FFF term with contractions all connecting different F.

# Integration by Parts

# Equations of Motion